# Check dates are uniform
## takes the first set of dates in a df filtered by filename and director name, and then checks that the dates in row[1:End] are the same as in row[0]

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas import ExcelWriter
import os
import numpy as np
from itertools import cycle, zip_longest

import pickle

from tqdm import tnrange, tqdm_notebook

from collections import Counter
import collections

from tkinter import *
from tkinter import messagebox

from datetime import datetime
import time
from datetime import date

import stat

from re import search

# constants

In [2]:
df_GRAND_SOURCE = r'C:\Users\Paul\backup_1Nov19\adjata\21_PICKLE_FILES\df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_24JUL20_III.pickle'

# functions

In [3]:
def open_pickle_file_as_df(path_and_fnme_string):
    
    pickle_in = open(path_and_fnme_string,"rb")
    df_pickle = pickle.load(pickle_in)
    
    return df_pickle

In [4]:
def saveFileAsPickle(path_and_fnme_string, file_to_save):
    
    pickle_out = open(path_and_fnme_string,"wb")
    pickle.dump(file_to_save, pickle_out)
    pickle_out.close()

In [5]:
def saveAsExcelFile(filename, df):
    
    options = {}
    options['strings_to_formulas'] = False
    options['strings_to urls'] = False

    file_name = filename.replace('.AX', "") + r'.xlsx'

    # file_to_save = data_folder / file_name

    file_to_save = file_name

    writer = ExcelWriter(file_to_save, options=options)
    df.to_excel(writer, 'title')
    writer.save()

In [6]:
def openExcelFile(file_name):

    workbook = pd.ExcelFile(file_name)

    d = {}

    for sheet_name in workbook.sheet_names:
        df = workbook.parse(sheet_name)
        d[sheet_name] = df
    
    return df

In [7]:
def open_pickle_file_and_save_as_excel_file(path_and_fnme_string_of_pickle_file, filename_to_save_as_excel_file):
    
    import os
    
    df_temp = open_pickle_file_as_df(path_and_fnme_string_of_pickle_file)
    
    filepath_filename_tuple = os.path.split(os.path.abspath(path_and_fnme_string_of_pickle_file))
    
    excel_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_to_save_as_excel_file.replace('.xlsx','')
    
    print(excel_filepath_and_filename)
    
    saveAsExcelFile(excel_filepath_and_filename, df_temp)

In [ ]:
# to create mirror of pickle file in use and save as Excel file for modifications
# open_pickle_file_and_save_as_excel_file(pickle_file_path_and_name_GRAND_SOURCE, 'test_Excel_file')

In [8]:
def open_excel_file_and_save_as_pickle_file(path_and_fnme_string_of_excel_file, filename_to_save_as_pickle_file):
    
    import os
    
    df_temp = openExcelFile(path_and_fnme_string_of_excel_file)
    
    print('opened Excel file')
    
    if 'Column1' in df_temp.columns:
    
        df_temp.drop(['Column1'], axis=1, inplace=True)

    if 'Unnamed: 0' in df_temp.columns:

        df_temp.drop(['Unnamed: 0'], axis=1, inplace=True)
        
    filepath_filename_tuple = os.path.split(os.path.abspath(path_and_fnme_string_of_excel_file))
    
    pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_to_save_as_pickle_file.replace('.pickle','') + r'.pickle'
    
    print(pickle_filepath_and_filename)
    
    saveFileAsPickle(pickle_filepath_and_filename, df_temp)

In [ ]:
# to open adjusted Excel file and save as pickle file
# open_excel_file_and_save_as_pickle_file(r'C:\Users\Paul\backup_1Nov19\python_stock_APIs\review_files\processing_completed_sub_files\split_files\df_SINGLE_3Z_II.xlsx', r'df_SINGLE_3Z_II.pickle')

In [9]:
def openPDFFileFromFileName(pdf_name):
    
    import subprocess
    import os

    doc_id = str(pdf_name)
    doc_ext_pdf = str('pdf')
    doc_ext_txt = str('txt')

    doc_id = doc_id.replace('.pdf', '').replace('.txt', '')

    # doc_path = str(r'C:\Users\paul_\Documents\adjata\1_data_testing_Feb19')
    doc_path = str(r'C:\Users\Paul\backup_1Nov19\adjata\1_data_testing_Feb19')
    doc_year = 2008

    while doc_year < 2019:

        path_to_file_to_open_pdf = doc_path + '\\' + 'COPIED_DOCUMENTS_' + str(doc_year) + '.' + doc_ext_txt
        path_to_file_to_open_txt = doc_path + '\\' + 'COPIED_DOCUMENTS_TEXT_' + str(doc_year) + '.' + doc_ext_txt

        cnt = 0

        with open(path_to_file_to_open_pdf, 'r') as fp:
            lines = fp.read().splitlines()
            for l in lines:
                if doc_id in l and len(l[-(len(l) - l.rfind('\\')) + 1:].replace('.pdf', '')) == len(doc_id):
                    cnt = cnt + 1
                    full_path_to_doc_to_open_pdf = l
                    full_path_to_doc_to_open_txt = full_path_to_doc_to_open_pdf.replace('.pdf', '.txt')

                    # replace Drive location
    #                 full_path_to_doc_to_open_pdf = full_path_to_doc_to_open_pdf.replace(r'E:', r'D:')
    #                 full_path_to_doc_to_open_txt = full_path_to_doc_to_open_txt.replace(r'E:', r'D:')

        if cnt > 0:
            print(doc_year, full_path_to_doc_to_open_pdf)
#             print(doc_year, '\n', full_path_to_doc_to_open_pdf, '\n', full_path_to_doc_to_open_txt)
            break

        doc_year = doc_year + 1

    # open pdf document
    path_to_pdf = os.path.abspath(full_path_to_doc_to_open_pdf)
    path_to_acrobat = os.path.abspath(r'C:\Program Files (x86)\Adobe\Acrobat Reader DC\Reader\AcroRd32.exe') 
    
    process = subprocess.Popen([path_to_acrobat, path_to_pdf], shell=False, stdout=subprocess.PIPE)

In [10]:
def reorderColumns(df_joined, column_list):

    # reorder columns

    df_new = df_joined[column_list] # reorders columns. Must be the same as the existing column names
    
    return df_new

In [11]:
def changeNamesInDFToReflectCorrectedNames(dict_type, dfToChange):
    
    # get document type 3X|3Y|3Z
    res = list(dict_type.keys())[0]
    docType = dict_type[res]['doc_type']

    df_filtered_for_doc_type = dfToChange[dfToChange['document_type'] == docType]

    for k, v in tqdm_notebook(dict_type.items(), total=len(dict_type), desc='main_loop'):

        filename_ = k
        
        for k1, v1 in v['directors'].items():
            
            director_name_round_three = k1
            director_name_round_four = v1
            
            # only if director_name_round_three and director_name_round_four are different do you go further
            if director_name_round_three != director_name_round_four:

                df_temp = df_filtered_for_doc_type[df_filtered_for_doc_type['filename'] == filename_]
                
                df_temp_director_three = df_temp[df_temp['director_name_round_three'] == director_name_round_three]
                
                index_list = df_temp_director_three.index.tolist()
                
                director_name_round_four_check_list = list(set(list(df_temp_director_three['director_name_round_four'])))
                
                if director_name_round_four_check_list[0] != v1:
                    
#                     if v1 == 'Peter Sisley Thomas':
                        
#                         print(k, index_list, director_name_round_four_check_list[0])
                    
                    docTypeIndex_dict = {'3X': 1, '3Y': 2, '3Z': 3}
                    
                    index_list = df_temp_director_three.index.tolist()

                    for i in range(0, len(index_list)):

                        index_ = index_list[i]

                        dfToChange.at[index_,'director_name_temp'] = director_name_round_four
                        dfToChange.at[index_,'director_name_corrected'] = docTypeIndex_dict[docType]
                                    
    return dfToChange

In [12]:
def convert_df_cell_string_to_date_object(cell_value):
    
    from datetime import date
    
    if not if_nan_return_True_otherwise_return_False(cell_value):
    
        dateObject = pd.to_datetime(cell_value).strftime("%d/%m/%Y")
        
        dateObject = datetime.strptime(dateObject, '%d/%m/%Y').date()

        return dateObject
    
    else:
        
        return date.today()
#         return cell_value

In [13]:
def convert_df_cell_string_to_date_object_npnan_if_false(cell_value):
    
    from datetime import date
    
    if not if_nan_return_True_otherwise_return_False(cell_value):
    
        dateObject = pd.to_datetime(cell_value).strftime("%d/%m/%Y")
        
        dateObject = datetime.strptime(dateObject, '%d/%m/%Y').date()

        return dateObject
    
    else:
        
        return np.nan
#         return cell_value

In [14]:
def performDataCheckUsingDirectorName(df_to_check, dir_name):

    df_test = df_to_check[df_to_check['director_name_round_four'] == dir_name]

    filename_list = list(set(list(df_test['filename'])))
    ticker_list = list(set(list(df_test['ticker'])))
    company_name_list = list(set(list(df_test['name_to_display'])))
    
    return filename_list, ticker_list, company_name_list

In [15]:
def if_nan_return_True_otherwise_return_False(x):
    
    if (x is np.nan or x != x):
        return True
    elif (isinstance(x, str) and (x.lower() == 'nan' or x.lower() == 'nat')):
        return True
    else:
        return False

# operations

In [16]:
df_check_dates = open_pickle_file_as_df(df_GRAND_SOURCE)
df_check_dates.reset_index(drop = True, inplace = True)

df_check_dates.head()

,filename,pdf_id,rec_id,file_date,document_type,rec_type0,rec_type,received_datetime,released_datetime,number_of_entities,exchange_id,ticker,entity_name,director_name,last_notice_date,cover_sheet_date,holder_name,interest_type,appointment_date,change_date,cessation_date,event_date,security_name_long,security_name_short,number,prior_number,acquired_number,disposed_number,after_number,change_nature,consideration,consideration_per_security,exercise_price,expiry_date,load_id,date_added,datetime,status,description,_merge_3X3Y3Z,_merge_load_log,unique_id,director_name_string,director_name_changed,director_name_changed_bool,ticker_reviewed_bool,entity_name_changed,entity_name_changed_bool,entity_name_reviewed_bool,director_name_final,director_name_final_bool,director_name_overall,director_name_overall_bool,alternative_name,director_name_round_two,director_name_round_two_bool,director_name_round_one,director_name_round_one_bool,entity_name_corrected,entity_name_final,entity_name_final_ticker_checked,entity_name_final_bool,orig_index_I,ASX_change_bool,ticker_changes,name_changes,last_ticker,last_name,ASX_change_dates,last_ASX_change_date,ticker_to_display,name_to_display,delisted_bool,delisted_date,orig_index_II,director_name_round_three,director_name_round_three_bool,director_name_round_four_old,director_name_round_four_bool,corrected_names_14Nov19_bool,director_name_round_four,director_name_corrected
0,470275.pdf,319266,70889.0,2019-02-06,3X,2008,NaN,2009-11-23 08:25:00,2009-11-23 08:25:00,1,0,EMS,Eastland Medical Systems Ltd,Calvin John ROSS,NaN,NaT,Calvin John Ross,Direct,2009-11-20,NaN,NaN,2009-11-20,Ordinary fully paid shares,Shares,1500000.0,0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 07:23:38,2019-07-05 07:20:56,success,NaN,both,both,470275.pdf31926670889.03X,Calvin John Ross,Calvin John Ross,0,1,Eastland Medical Systems Ltd,0,0,Calvin John Ross,0,Calvin John Ross,0,NaN,Calvin John Ross,0,Calvin John Ross,0,EASTLAND MEDICAL SYSTEMS LIMITED,EASTLAND MEDICAL SYSTEMS LIMITED,0,0,1173,1,"EMS, SUD, SUD, SUD","EASTLAND MEDICAL SYSTEMS LIMITED, SUDA LIMITED...",SUD,SUDA PHARMACEUTICALS LIMITED,NaT,2017-12-05,SUD,SUDA PHARMACEUTICALS LIMITED,0,NaN,1173,Calvin John Ross,0,Calvin John Ross,0,0,Calvin John Ross,0
1,6508702.pdf,445323,99694.0,2019-02-07,3X,2008,NaN,2011-09-29 19:01:00,2011-09-29 19:01:00,1,0,SHC,"Sunshine Heart, Inc.",Mark Harvey,NaN,NaT,NaN,NaN,2011-09-29,NaN,NaN,2011-09-29,NaN,NaN,0.0,0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:27:05,2019-07-05 07:22:12,success,NaN,both,both,6508702.pdf44532399694.03X,Mark Harvey,Mark Harvey,0,1,Sunshine Heart Inc,1,0,Mark Harvey,0,Mark Harvey,0,NaN,Mark Harvey,0,Mark Harvey,0,SUNSHINE HEART INC,SUNSHINE HEART INC,0,0,229485,0,NaN,NaN,NaN,NaN,NaT,NaT,SHC_,SUNSHINE HEART INC,1,2013-05-06 00:00:00,229471,Mark Harvey,0,Mark Harvey,0,0,Mark Harvey,0
2,6513462.pdf,461688,103389.0,2019-07-05,3X,2008,NaN,2011-10-04 11:09:00,2011-10-04 11:09:00,1,0,XAM,XANADU MINES LTD,Hannah Louise Badenach,NaN,2011-10-04,NaN,NaN,2011-10-04,NaN,NaN,2011-10-04,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:52:22,2019-07-05 07:16:23,success,NaN,both,both,6513462.pdf461688103389.03X,Hannah Louise Badenach,Hannah Louise Badenach,0,1,XANADU MINES LTD,0,0,Hannah Louise Badenach,0,Hannah Louise Badenach,0,NaN,Hannah Louise Badenach,0,Hannah Louise Badenach,0,XANADU MINES LIMITED,XANADU MINES LIMITED,0,0,259059,0,NaN,NaN,NaN,NaN,NaT,NaT,XAM,XANADU MINES LIMITED,0,NaN,259043,Hannah Louise Badenach,0,Hannah Louise Badenach,0,0,Hannah Louise Badenach,0
3,01974250.pdf,442406,99059.0,2019-02-05,3X,2008,NaN,2018-04-24 15:58:06,2018-04-24 15:58:06,1,6,MDD,MANDALONG RESOURCES LIMITED,HONGJUN CUI,NaN,NaT,NaN,NaN,2018-04-24,NaN,NaN,2018-04-24,NaN,NaN,0.0,0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:22:55,2019-07-05 07:18:50,success,NaN,both,both,01974250.pdf44240699059.03X,H

In [17]:
performDataCheckUsingDirectorName(df_check_dates, 'John M Harrison')[1:3]

(['EMB'], ['EMBELTON LIMITED'])

In [ ]:
xxxxx

# loop through df and check dates - first main dictionary generator

In [ ]:
director_name_list

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

check_dates_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_temp = df_check_dates[df_check_dates['filename'] == ele]
    
    director_name_list = list(set(list(df_temp['director_name_round_four'])))
    
    for idxx, elex in enumerate(director_name_list):
        
        df_temp_director = df_temp[df_temp['director_name_round_four'] == elex]
        
        index_error_list = [] 
        
        if df_temp_director.shape[0] > 0:
            
            index_list = df_temp_director.index.tolist()
            
            index_list_sorted = sorted(index_list)
            
            first_row = index_list_sorted[0]
            
            cover_sheet_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cover_sheet_date'])
            last_notice_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'last_notice_date'])
            appointment_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'appointment_date'])
            cessation_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cessation_date'])
            change_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'change_date'])
            event_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'event_date'])
            
            first_row_date_list = [
                cover_sheet_date,
                last_notice_date,
                appointment_date,
                cessation_date,
                change_date,
                event_date
            ]
            
            for idxxx, elexxx in enumerate(index_list_sorted):
                
                if idxxx > 0:
                    
                    cover_sheet_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'cover_sheet_date'])
                    last_notice_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'last_notice_date'])
                    appointment_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'appointment_date'])
                    cessation_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'cessation_date'])
                    change_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'change_date'])
                    event_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'event_date'])
                    
                    subsequent_row_date_list = [
                        cover_sheet_date,
                        last_notice_date,
                        appointment_date,
                        cessation_date,
                        change_date,
                        event_date
                    ]
                    
                    for i in range(0, len(first_row_date_list)):
                        
                        director_name_error_list = []
                        
                        sub_director_dict = {}
                        
                        if first_row_date_list[i] != subsequent_row_date_list[i]:
                            
                            index_error_list.append(first_row)
                            index_error_list.append(elexxx)
                            index_error_list = list(set(index_error_list))
                            
#                             print(first_row_date_list[i], subsequent_row_date_list[i])
                            
                            key = str(ele) + r'_' + str(elex)
                            
                            director_name_error_list.append(elex)
                            
                            director_name_error_list = list(set(director_name_error_list))
                            
                            if key not in check_dates_dict:
                                
                                check_dates_dict.setdefault(key, [])
                                
                            sub_director_dict.setdefault(first_row, [])
                            
                            sub_director_dict[first_row] = elex
                            
                            index_error_list_sorted = sorted(index_error_list)
                            
                            # get doc_types
                            
                            df_doc_types = df_check_dates[
                                (df_check_dates['filename'] == ele)
                                &
                                (df_check_dates['director_name_round_four'] == elex)
                                &
                                (df_check_dates.index.isin(index_error_list_sorted))

                            ]
                            
                            doc_types = list(set(list(df_doc_types['document_type'])))
                            
                            check_dates_dict[key] = {'filename': ele, 'director': sub_director_dict[first_row], 'first_row': first_row, 'complete_list': index_error_list_sorted, 'document_types': doc_types}
                            
#                             print('different', ele, check_dates_dict[key])
                    
                    

In [ ]:
check_dates_dict

In [ ]:
fnme = '232779.pdf'
dir_name = 'Paul Lin'
row_list = [253, 34295]

df_check_dates[
    (df_check_dates['filename'] == fnme)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
#     &
#     (df_check_dates.index.isin(row_list))
    
]

In [ ]:
xxxxx

In [ ]:
check_dates_dict

In [ ]:
# SAVE date error dictionary

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'date_errors_across_filtered_rows_dict_III_12JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, check_dates_dict)

In [ ]:
XXXX

# open saved dictionary pickle file

In [ ]:
filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'date_errors_across_filtered_rows_dict_III_12JUL20.pickle'

check_dates_dict = open_pickle_file_as_df(pickle_filepath_and_filename)

In [ ]:
check_dates_dict

# 1. first check if different date is cover sheet date

In [ ]:
dict_key_list = []

for k, v in check_dates_dict.items():
    
    dict_key_list.append(k)

In [ ]:
all_errors_list = []

for idx, ele in enumerate(dict_key_list):
    
    detected_error_row_list = []
    
    filename_ = check_dates_dict[ele]['filename']
    director_name = check_dates_dict[ele]['director']
    error_row_list = check_dates_dict[ele]['complete_list']
    
    df_temp = df_check_dates[(df_check_dates['filename'] == filename_)
                            &
                            (df_check_dates['director_name_round_four'] == director_name)
                            &
                            (df_check_dates.index.isin(error_row_list))
                            ]
    
    first_row = error_row_list[0]
    
    cover_sheet_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cover_sheet_date'])
    
    for idxx, elex in enumerate(error_row_list):
        
        if idxx > 0:
            
            cover_sheet_date_subs_row = convert_df_cell_string_to_date_object(df_check_dates.loc[elex, 'cover_sheet_date'])
            
            if cover_sheet_date_first_row != cover_sheet_date_subs_row:
                
                detected_error_row_list.append(first_row)
                detected_error_row_list.append(elex)
                detected_error_row_list = list(set(detected_error_row_list))
                
    if len(detected_error_row_list) > 0:

        print(ele, filename_, director_name, detected_error_row_list) #first_row, elex)
        
        temp_list = [ele, filename_, director_name, detected_error_row_list]
        
        all_errors_list.append(temp_list)
            
print('analysis completed')           

In [ ]:
index_to_use = 0
to_use_list = all_errors_list[index_to_use]
to_use_list[1], to_use_list[2], to_use_list[3], len(all_errors_list)

In [ ]:
filename_ = to_use_list[1]
director_name = to_use_list[2]
error_row_list = to_use_list[3]

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == director_name)
    &
    (df_check_dates.index.isin(error_row_list))
    ]



In [ ]:
index_to_correct = 10016
cover_sheet_date_to_use = '2010-09-01'

if df_check_dates.loc[index_to_correct, 'director_name_round_four'] == director_name:

    df_check_dates.at[index_to_correct, 'cover_sheet_date'] = cover_sheet_date_to_use
    
    print('change made for', filename_, director_name, index_to_correct)
    
else:
    
    print('THERE IS AN ERROR!!!')

# 2. second check, check all other dates, but split by doc type

In [ ]:
all_errors_list = []

date_type_dict = {0: 'cover_sheet_date',
                  1: 'last_notice_date',
                  2: 'appointment_date',
                  3: 'change_date',
                  4: 'cessation_date',
                  5: 'event_date'}

for idx, ele in enumerate(dict_key_list):
    
    filename_ = check_dates_dict[ele]['filename']
    director_name = check_dates_dict[ele]['director']
    error_row_list = check_dates_dict[ele]['complete_list']
    doc_type_list = check_dates_dict[ele]['document_types']
    
    for idxx, elex in enumerate(doc_type_list):
        
        error_row_detected_list = []
        date_error_type = []
    
        df_temp = df_check_dates[(df_check_dates['filename'] == filename_)
                                &
                                (df_check_dates['director_name_round_four'] == director_name)
                                &
                                (df_check_dates['document_type'] == elex)
                                &
                                (df_check_dates.index.isin(error_row_list))
                                ]

        if df_temp.shape[0] > 1:
            
#             print(idxx, elex)
            
            temp_row_list = df_temp.index.tolist()
            temp_row_list_sorted = sorted(temp_row_list)
        
            first_row = temp_row_list_sorted[0]

            cover_sheet_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cover_sheet_date'])
            last_notice_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'last_notice_date'])
            appointment_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'appointment_date'])
            cessation_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cessation_date'])
            change_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'change_date'])
            event_date = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'event_date'])

            first_row_date_list = [
                cover_sheet_date,
                last_notice_date,
                appointment_date,
                cessation_date,
                change_date,
                event_date
            ]

            for idxxx, elexx in enumerate(temp_row_list_sorted):

                if idxxx > 0:
                    
#                     print(idxx, elex)

                    cover_sheet_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'cover_sheet_date'])
                    last_notice_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'last_notice_date'])
                    appointment_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'appointment_date'])
                    cessation_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'cessation_date'])
                    change_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'change_date'])
                    event_date = convert_df_cell_string_to_date_object(df_check_dates.loc[elexx, 'event_date'])

                    subsequent_row_date_list = [
                        cover_sheet_date,
                        last_notice_date,
                        appointment_date,
                        cessation_date,
                        change_date,
                        event_date
                    ]

                    for idxxxx, elexxx in enumerate(first_row_date_list):

                        if first_row_date_list[idxxxx] != subsequent_row_date_list[idxxxx]:

                            error_row_detected_list.append(first_row)
                            error_row_detected_list.append(elexx)
                            error_row_detected_list = list(set(error_row_detected_list))

                            error_row_detected_list_sorted = sorted(error_row_detected_list)
                            
                            date_error_type.append(date_type_dict[idxxxx])
                            date_error_type = list(set(date_error_type))

    if len(error_row_detected_list) > 0:

        print(ele, filename_, director_name, error_row_detected_list, date_error_type) #first_row, elex)
        
        temp_list = [ele, filename_, director_name, error_row_detected_list, date_error_type]
        
        all_errors_list.append(temp_list)
            
print('analysis completed')         

In [ ]:
index_to_use = 0
to_use_list = all_errors_list[index_to_use]
to_use_list[1], to_use_list[2], to_use_list[3], to_use_list[4], len(all_errors_list)

In [ ]:
filename_ = to_use_list[1]
director_name = to_use_list[2]
error_row_list = to_use_list[3]

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == director_name)
    &
    (df_check_dates.index.isin(error_row_list))
    ]

In [ ]:
date_type_dict = {0: 'cover_sheet_date',
                  1: 'last_notice_date',
                  2: 'appointment_date',
                  3: 'change_date',
                  4: 'cessation_date',
                  5: 'event_date'}

index_to_correct = 33128
date_to_use = '2011-08-10'

if df_check_dates.loc[index_to_correct, 'director_name_round_four'] == director_name:

    df_check_dates.at[index_to_correct, 'event_date'] = date_to_use
    
    print('change made for', filename_, director_name, index_to_correct, 'to', date_to_use)
    
else:
    
    print('THERE IS AN ERROR!!!')

# save corrected df

In [ ]:
# SAVE date error dictionary

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_12JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# to do:
1. run first dictionary generator to check work - done
2. filter for filename only and check for cover sheet date uniformity
3. filter for 3X & 3Z and check event date is not nan

In [ ]:
# start from here

# 3. filter for filename only and check for cover sheet date uniformity

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

check_dates_filename_and_coversheet_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_temp = df_check_dates[df_check_dates['filename'] == ele]
        
    index_error_list = []

    if df_temp.shape[0] > 0:

        index_list = df_temp.index.tolist()

        index_list_sorted = sorted(index_list)

        first_row = index_list_sorted[0]

        cover_sheet_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cover_sheet_date'])

        for idxxx, elexxx in enumerate(index_list_sorted):

            if idxxx > 0:

                cover_sheet_date_subs_row = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'cover_sheet_date'])

                if cover_sheet_date_first_row != cover_sheet_date_subs_row:

#                     index_error_list.append(first_row)
#                     index_error_list.append(elexxx)
                    for x in index_list:
                        index_error_list.append(x)
                    
                    index_error_list = list(set(index_error_list))

#                             print(first_row_date_list[i], subsequent_row_date_list[i])

                    key = ele

                    if key not in check_dates_filename_and_coversheet_dict:

                        check_dates_filename_and_coversheet_dict.setdefault(key, [])

                    index_error_list_sorted = sorted(index_error_list)

                    check_dates_filename_and_coversheet_dict[key] = {'filename': ele, 'first_row': first_row, 'complete_list': index_error_list_sorted}

#                             print('different', ele, check_dates_dict[key])

In [ ]:
check_dates_filename_and_coversheet_dict

In [ ]:
# check_dates_filename_and_coversheet_dict

In [ ]:
dict_key_list = []

for k, v in check_dates_filename_and_coversheet_dict.items():
    
    dict_key_list.append(k)

In [ ]:
index_to_use = 0
filename_ = dict_key_list[index_to_use]
error_row_list = check_dates_filename_and_coversheet_dict[filename_]['complete_list']
filename_, error_row_list, len(dict_key_list)

In [ ]:
openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates.index.isin(error_row_list))
    ]

In [ ]:
index_to_correct_list = [29895, 29896]
index_zero = index_to_correct_list[0]
date_to_use = '2009-02-12'

# test that all rows have the correct filename
correct_row_bool_list = []

for i in range(0, len(index_to_correct_list)):
    
    if df_check_dates.loc[index_to_correct_list[i], 'filename'] == filename_:
        
        correct_row_bool_list.append(True)
        
    else:
        
        correct_row_bool_list.append(True)
        
correct_row_bool_list = list(set(correct_row_bool_list))

# if df_check_dates.loc[index_zero, 'filename'] == filename_:

if len(correct_row_bool_list) == 1 and correct_row_bool_list[0] == True:

    for i in range(0, len(index_to_correct_list)):
        
        df_check_dates.at[index_to_correct_list[i], 'cover_sheet_date'] = date_to_use
    
        print('change made for', filename_, '; index:', index_to_correct_list[i], '; to:', date_to_use)
    
else:
    
    print('THERE IS AN ERROR!!!')

# save corrected df

In [ ]:
# SAVE date error dictionary

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_13JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# 4. filter for filename  & [3X | 3Z] only and check for existence of event_date and:
1. appointment_date == event_date [3X]
2. cessation_date == event_date [3Z]

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

doc_type_list = ['3X','3Z']

check_3X_3Z_dates_filename_and_coversheet_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_temp = df_check_dates[df_check_dates['filename'] == ele]
    
    director_name_list = list(set(list(df_temp['director_name_round_four'])))
        
#     index_error_list = []
    
    for ii in range(0, len(director_name_list)):
        
        index_error_list = []
        
        dir_name = director_name_list[ii]
        
        df_temp_dir_name = df_temp[df_temp['director_name_round_four'] == dir_name]
    
        # filter for doc type

        for idxx, elexx in enumerate(doc_type_list):

            df_temp_doc_type = df_temp_dir_name[df_temp_dir_name['document_type'] == elexx]

            if df_temp_doc_type.shape[0] > 0:

                index_list = df_temp_doc_type.index.tolist()

                index_list_sorted = sorted(index_list)

                first_row = index_list_sorted[0]

                if elexx == '3X':

                    comparison_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'appointment_date'])

                elif elexx == '3Z':

                    comparison_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cessation_date'])

                event_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'event_date'])

                first_row_date_list = [
                    comparison_date_first_row,
                    event_date_first_row
                ]

                for idxxx, elexxx in enumerate(index_list_sorted):

                    if idxxx > 0:

                        if elexx == '3X':

                            comparison_date_subs_row = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'appointment_date'])

                        elif elexx == '3Z':

                            comparison_date_subs_row = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'cessation_date'])

                        event_date_subs_row = convert_df_cell_string_to_date_object(df_check_dates.loc[elexxx, 'event_date'])

                        subsq_row_date_list = [
                            comparison_date_subs_row,
                            event_date_subs_row
                        ]

                        for i in range(0, len(first_row_date_list)):

                            if (first_row_date_list[i] != subsq_row_date_list[i]) or (comparison_date_first_row != event_date_first_row) or (comparison_date_subs_row != event_date_subs_row):

    #                             index_error_list.append(first_row)
    #                             index_error_list.append(elexxx)

                                for x in index_list_sorted:
                                    index_error_list.append(x)

                                index_error_list = list(set(index_error_list))

            #                             print(first_row_date_list[i], subsequent_row_date_list[i])

                                key = str(ele) + r'_' + dir_name.replace(' ', '_')

                                if key not in check_3X_3Z_dates_filename_and_coversheet_dict:

                                    check_3X_3Z_dates_filename_and_coversheet_dict.setdefault(key, [])

                                index_error_list_sorted = sorted(index_error_list)

                                check_3X_3Z_dates_filename_and_coversheet_dict[key] = {'filename': ele, 'director_name': dir_name, 'doc_type': elexx, 'complete_list': index_error_list_sorted}
                                
#                                 if ele == '01792355.pdf':
                                    
#                                     print(key, check_3X_3Z_dates_filename_and_coversheet_dict[key])

#                                 print('difference recorded', key)

In [ ]:
# save check_3X_3Z_dates_filename_and_coversheet_dict.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'check_3X_3Z_dates_filename_and_coversheet_dict_14JUL20_b.pickle'

saveFileAsPickle(pickle_filepath_and_filename, check_3X_3Z_dates_filename_and_coversheet_dict)

In [ ]:
dict_key_list = []

for k, v in check_3X_3Z_dates_filename_and_coversheet_dict.items():
    
    dict_key_list.append(k)

In [ ]:
dict_key_list_SHORTER = dict_key_list #[240:254]

In [ ]:
# filenames_completed_list = []

In [ ]:
dict_key_list_SHORTER

In [ ]:
filenames_completed_list

In [ ]:
# start here AGAIN!

for idx, elex in enumerate(dict_key_list_SHORTER):
    
    print(idx, elex)
    
    if elex not in filenames_completed_list:

        index_to_use = idx
        key_for_dict = elex
        filename_ = check_3X_3Z_dates_filename_and_coversheet_dict[key_for_dict]['filename']
        doc_type_in_dict = check_3X_3Z_dates_filename_and_coversheet_dict[key_for_dict]['doc_type']
        dir_name = check_3X_3Z_dates_filename_and_coversheet_dict[key_for_dict]['director_name']
        error_row_list = check_3X_3Z_dates_filename_and_coversheet_dict[key_for_dict]['complete_list']

        index_to_correct_list = error_row_list

        doc_type_in_use = '3X'

        if doc_type_in_use == doc_type_in_dict:

            df_to_get_list_of_dates = df_check_dates[
                (df_check_dates['filename'] == filename_)
                &
                (df_check_dates['document_type'] == doc_type_in_use)
                &
                (df_check_dates['director_name_round_four'] == dir_name)
                &
                (df_check_dates.index.isin(error_row_list))
            ]

            if doc_type_in_use == '3X':

                list_of_dates_in_df_list = list(set(list(df_to_get_list_of_dates['appointment_date'])))

            else:

                list_of_dates_in_df_list = list(set(list(df_to_get_list_of_dates['cessation_date'])))

            # get list of dates currently used in df

            openPDFFileFromFileName(filename_)

            print(filename_, doc_type_in_use, '____', dir_name, '____', error_row_list, idx, len(dict_key_list_SHORTER), len(filenames_completed_list), len(dict_key_list), list_of_dates_in_df_list)

            date_to_use = input('What date do you want to use')

            substring = r"^[\-]{2}"

            if search(substring, date_to_use):
                date_to_use = list_of_dates_in_df_list[0]
            else:
                date_to_use = date_to_use

            # test that all rows have the correct filename
            
            #### AND ADD IN DIRECTOR NAME AS WELL ####
            correct_row_bool_list = []

            for i in range(0, len(index_to_correct_list)):

                if df_check_dates.loc[index_to_correct_list[i], 'filename'] == filename_ and df_check_dates.loc[index_to_correct_list[i], 'director_name_round_four'] == dir_name:

                    correct_row_bool_list.append(True)

                else:

                    correct_row_bool_list.append(True)

            correct_row_bool_list = list(set(correct_row_bool_list))

            # if df_check_dates.loc[index_zero, 'filename'] == filename_:

            if len(correct_row_bool_list) == 1 and correct_row_bool_list[0] == True:
                
                filenames_completed_list.append(elex)

                for i in range(0, len(index_to_correct_list)):

                    if doc_type_in_use == '3X':

                        df_check_dates.at[index_to_correct_list[i], 'appointment_date'] = date_to_use
                        df_check_dates.at[index_to_correct_list[i], 'event_date'] = date_to_use

                        print('change made for', filename_, '; director:', dir_name, 'index:', index_to_correct_list[i], '; to:', date_to_use)

                    elif doc_type_in_use == '3Z':

                        df_check_dates.at[index_to_correct_list[i], 'cessation_date'] = date_to_use
                        df_check_dates.at[index_to_correct_list[i], 'event_date'] = date_to_use

                        print('change made for', filename_, '; director:', dir_name, 'index:', index_to_correct_list[i], '; to:', date_to_use)

            else:

                print('THERE IS AN ERROR!!!')
            
print('all done!')
    

In [ ]:
# 721220.pdf_Alexander_James_Neuling_3Z

df_check_dates[
#     (df_check_dates['document_type'] == '3Z')
#     &
    (df_check_dates['filename'] == '721220.pdf')
]

# save corrected df

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_14JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# 5. rework No 1

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

doc_type_list = ['3X','3Z']

check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_temp = df_check_dates[df_check_dates['filename'] == ele]
    
    director_name_list = list(set(list(df_temp['director_name_round_four'])))
        
#     index_error_list = []
    
    for ii in range(0, len(director_name_list)):
        
        index_error_list = []
        
        dir_name = director_name_list[ii]
        
        df_temp_dir_name = df_temp[df_temp['director_name_round_four'] == dir_name]
    
        # filter for doc type

        for idxx, elexx in enumerate(doc_type_list):

            df_temp_doc_type = df_temp_dir_name[df_temp_dir_name['document_type'] == elexx]

            if df_temp_doc_type.shape[0] > 0:

                index_list = df_temp_doc_type.index.tolist()

                index_list_sorted = sorted(index_list)

                first_row = index_list_sorted[0]

                if elexx == '3X':

                    comparison_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'appointment_date'])

                elif elexx == '3Z':

                    comparison_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'cessation_date'])

                event_date_first_row = convert_df_cell_string_to_date_object(df_check_dates.loc[first_row, 'event_date'])

                

                if (comparison_date_first_row != event_date_first_row) or if_nan_return_True_otherwise_return_False(comparison_date_first_row) or if_nan_return_True_otherwise_return_False(event_date_first_row):

#                             index_error_list.append(first_row)
#                             index_error_list.append(elexxx)

                    for x in index_list_sorted:
                        index_error_list.append(x)

                    index_error_list = list(set(index_error_list))

#                             print(first_row_date_list[i], subsequent_row_date_list[i])

                    key = str(ele) + r'_' + dir_name.replace(' ', '_') + r'_' + elexx

                    if key not in check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict:

                        check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict.setdefault(key, [])

                    index_error_list_sorted = sorted(index_error_list)

                    check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict[key] = {'filename': ele, 'director_name': dir_name, 'doc_type': elexx, 'complete_list': index_error_list_sorted}

#                                 if ele == '01792355.pdf':

#                                     print(key, check_3X_3Z_dates_filename_and_coversheet_dict[key])

#                     print('difference recorded', key)

# save rework dictionary

In [ ]:
# save check_3X_3Z_dates_filename_and_coversheet_dict.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict_14JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict)

# REWORK 1 CORRECTIONS

In [ ]:
dict_key_list = []

for k, v in check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict.items():
    
    dict_key_list.append(k)

In [ ]:
dict_key_list_SHORTER = dict_key_list #[250:300]

In [ ]:
# filenames_completed_list = []

In [ ]:
# start here AGAIN!

for idx, elex in enumerate(dict_key_list_SHORTER):
    
    print(idx, elex)
    
    if elex not in filenames_completed_list:

        index_to_use = idx
        key_for_dict = elex
        filename_ = check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict[key_for_dict]['filename']
        doc_type_in_dict = check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict[key_for_dict]['doc_type']
        dir_name = check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict[key_for_dict]['director_name']
        error_row_list = check_3X_3Z_dates_filename_and_coversheet_REWORK_1_dict[key_for_dict]['complete_list']

        index_to_correct_list = error_row_list

        doc_type_in_use = '3Z'

        if doc_type_in_use == doc_type_in_dict:

            df_to_get_list_of_dates = df_check_dates[
                (df_check_dates['filename'] == filename_)
                &
                (df_check_dates['document_type'] == doc_type_in_use)
                &
                (df_check_dates['director_name_round_four'] == dir_name)
                &
                (df_check_dates.index.isin(error_row_list))
            ]

            if doc_type_in_use == '3X':

                list_of_dates_in_df_list = list(set(list(df_to_get_list_of_dates['appointment_date'])))

            else:

                list_of_dates_in_df_list = list(set(list(df_to_get_list_of_dates['cessation_date'])))

            # get list of dates currently used in df

            openPDFFileFromFileName(filename_)

            print(filename_, doc_type_in_use, '____', dir_name, '____', error_row_list, idx, len(dict_key_list_SHORTER), len(filenames_completed_list), len(dict_key_list), list_of_dates_in_df_list)

            date_to_use = input('What date do you want to use')

            substring = r"^[\-]{2}"

            if search(substring, date_to_use):
                date_to_use = list_of_dates_in_df_list[0]
            else:
                date_to_use = date_to_use

            # test that all rows have the correct filename
            
            #### AND ADD IN DIRECTOR NAME AS WELL ####
            correct_row_bool_list = []

            for i in range(0, len(index_to_correct_list)):

                if df_check_dates.loc[index_to_correct_list[i], 'filename'] == filename_ and df_check_dates.loc[index_to_correct_list[i], 'director_name_round_four'] == dir_name:

                    correct_row_bool_list.append(True)

                else:

                    correct_row_bool_list.append(True)

            correct_row_bool_list = list(set(correct_row_bool_list))

            # if df_check_dates.loc[index_zero, 'filename'] == filename_:

            if len(correct_row_bool_list) == 1 and correct_row_bool_list[0] == True:
                
                filenames_completed_list.append(elex)

                for i in range(0, len(index_to_correct_list)):

                    if doc_type_in_use == '3X':

                        df_check_dates.at[index_to_correct_list[i], 'appointment_date'] = date_to_use
                        df_check_dates.at[index_to_correct_list[i], 'event_date'] = date_to_use

                        print('change made for', filename_, '; director:', dir_name, 'index:', index_to_correct_list[i], '; to:', date_to_use)

                    elif doc_type_in_use == '3Z':

                        df_check_dates.at[index_to_correct_list[i], 'cessation_date'] = date_to_use
                        df_check_dates.at[index_to_correct_list[i], 'event_date'] = date_to_use

                        print('change made for', filename_, '; director:', dir_name, 'index:', index_to_correct_list[i], '; to:', date_to_use)

            else:

                print('THERE IS AN ERROR!!!')
            
print('all done!')

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_14JUL20_a.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# Final check

In [ ]:
# TEST 1: check that all coversheet dates are the same for the filename
# TEST 2: check that all appointment dates and event dates are the same for the same filename, director and 3X
# TEST 3: check that all cessation dates and event dates are the same for the same filename, director and 3Z

filename_list = list(set(list(df_check_dates['filename'])))
number_of_rows_in_df = df_check_dates.shape[0]

doc_type_list = ['3X', '3Y', '3Z']

final_check_dict = {}

# TEST 1: check that all coversheet dates are the same for the filename

cover_sheet_date_test_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='first_test'):
    
    df_filtered_by_filename = df_check_dates[df_check_dates['filename'] == ele]
    
    coversheet_date_list = df_filtered_by_filename['cover_sheet_date']
    
    coversheet_date_list_tested = [ if_nan_return_True_otherwise_return_False(x) for x in coversheet_date_list ]
    
    coversheet_date_list_tested = list(set(coversheet_date_list_tested))
    
    if len(coversheet_date_list_tested) > 1:
        
        key = ele
        
        cover_sheet_date_test_dict.setdefault(key, [])
        
        cover_sheet_date_test_dict[key] = {'cover_sheet_dates': coversheet_date_list}
    
#         print(cover_sheet_date_test_dict[key])

print('length of cover_sheet_date_test_dict is', len(cover_sheet_date_test_dict))

In [ ]:
# TEST 2: check that all appointment dates and event dates are the same for the same filename, director and 3X

appointment_date_3X_test_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='second_test'):
    
    df_filtered_by_filename = df_check_dates[df_check_dates['filename'] == ele]
    
    df_filtered_by_3X = df_filtered_by_filename[df_filtered_by_filename['document_type'] == '3X']
    
    dir_names_list = list(set(list(df_filtered_by_3X['director_name_round_four'])))
    
    for idxx, elexx in enumerate(dir_names_list):
        
        # filter for director name
        
        df_filtered_by_dir_name = df_filtered_by_3X[df_filtered_by_3X['director_name_round_four'] == elexx]
        
        # check for only one appointment date, event date and that they are both the same
        
        appt_date = list(set(list(df_filtered_by_dir_name['appointment_date'])))
        event_date = list(set(list(df_filtered_by_dir_name['event_date'])))
        
        if (len(appt_date) > 1) or (len(event_date) > 1) or (appt_date != event_date):
            
            key = ele + r'_' + elexx + r'_' + '3X'
            
            appointment_date_3X_test_dict.setdefault(key, [])
            
            appointment_date_3X_test_dict[key] = {'filename': ele, 'director': elexx, 'document_type': '3X'}
            
print('length of appointment_date_3X_test_dict is', len(appointment_date_3X_test_dict))
    

In [ ]:
# TEST 3: check that all cessation dates and event dates are the same for the same filename, director and 3Z

cessation_date_3Z_test_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='third_test'):
    
    df_filtered_by_filename = df_check_dates[df_check_dates['filename'] == ele]
    
    df_filtered_by_3Z = df_filtered_by_filename[df_filtered_by_filename['document_type'] == '3Z']
    
    dir_names_list = list(set(list(df_filtered_by_3Z['director_name_round_four'])))
    
    for idxx, elexx in enumerate(dir_names_list):
        
        # filter for director name
        
        df_filtered_by_dir_name = df_filtered_by_3Z[df_filtered_by_3Z['director_name_round_four'] == elexx]
        
        # check for only one cessation date, event date and that they are both the same
        
        cess_date = list(set(list(df_filtered_by_dir_name['cessation_date'])))
        event_date = list(set(list(df_filtered_by_dir_name['event_date'])))
        
        if (len(cess_date) > 1) or (len(event_date) > 1) or (cess_date != event_date):
            
            key = ele + r'_' + elexx + r'_' + '3Z'
            
            cessation_date_3Z_test_dict.setdefault(key, [])
            
            cessation_date_3Z_test_dict[key] = {'filename': ele, 'director': elexx, 'document_type': '3Z'}   
            
print('length of cessation_date_3Z_test_dict is', len(cessation_date_3Z_test_dict))

In [ ]:
print('length of cover_sheet_date_test_dict is', len(cover_sheet_date_test_dict))

In [ ]:
cessation_date_3Z_test_dict

In [ ]:
filename_ = '6093382.pdf'
doc_type = '3Z'
dir_name = 'Tony Harwood'

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['document_type'] == doc_type)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
]

In [ ]:
index_to_correct = 37054

corrected_date = '2012-04-18'

if df_check_dates.loc[index_to_correct, 'director_name_round_four'] == dir_name:

    if doc_type == '3X':
        df_check_dates.at[index_to_correct, 'appointment_date'] = corrected_date
    elif doc_type == '3Z':
        df_check_dates.at[index_to_correct, 'cessation_date'] = corrected_date
    
    df_check_dates.at[index_to_correct, 'event_date'] = corrected_date
    
    print('correction made')
    
else:
    
    print('THERE IS AN ERROR')

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_16JUL20.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# random check

In [ ]:
df_check_dates.shape[0]

In [ ]:
# generate random integer values
from random import seed
from random import randint
# seed random number generator
seed(1)
# generate some integers
for _ in range(10):
    index_ = randint(0, df_check_dates.shape[0])
    
    filename_ = df_check_dates.loc[index_, 'filename']
    dir_name = df_check_dates.loc[index_, 'director_name_round_four']
    doc_type = df_check_dates.loc[index_, 'document_type']
    
    if doc_type == '3X' or doc_type == '3Z':
    
        cover_sheet_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'cover_sheet_date'])
        last_notice_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'last_notice_date'])
        appointment_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'appointment_date'])
        cessation_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'cessation_date'])
        change_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'change_date'])
        event_date = convert_df_cell_string_to_date_object_npnan_if_false(df_check_dates.loc[index_, 'event_date'])

        openPDFFileFromFileName(filename_)
        
        result_string = 'filename:' + filename_ + "\n" + 'director_name:' + dir_name + "\n" + 'cover_sheet_date:' + cover_sheet_date #+ r"\n" + 'last_notice_date:' + last_notice_date + r"\n" + 'appointment_date:' + appointment_date + r"\n" + 'change_date:' + change_date + r"\n" + 'cessation_date:' + cessation_date + r"\n" + 'event_date:' + event_date

#         print(filename_, dir_name, doc_type, cover_sheet_date, last_notice_date, appointment_date, cessation_date, change_date, event_date)
        print(result_string)
        
        wait = input('next one')
    
print('Done')

# record number check
1. to see how many records each filename | director name | document type has
2. eg Andrew Tsang has 40 records - this is 4 x 10 correct records - this error somehow arose during the document review stage

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))
number_of_rows_in_df = df_check_dates.shape[0]

doc_type_list = ['3X', '3Y', '3Z']

record_number_check_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_filtered_by_filename = df_check_dates[df_check_dates['filename'] == ele]
    
    dir_name_list = list(set(list(df_filtered_by_filename['director_name_round_four'])))
    
    for idxx, elexx in enumerate(dir_name_list):
        
        df_filtered_by_dir_name = df_filtered_by_filename[df_filtered_by_filename['director_name_round_four'] == elexx]
        
        for idxxx, elexxx in enumerate(doc_type_list):
            
            df_filtered_by_doc_type = df_filtered_by_dir_name[df_filtered_by_dir_name['document_type'] == elexxx]
            
            if df_filtered_by_doc_type.shape[0] > 0:
                
                key = ele + r'_' + elexx.replace(' ', '_') + r'_' + elexxx
                
                count_ = df_filtered_by_doc_type.shape[0]
                
                record_number_check_dict.setdefault(key, [])
                
                record_number_check_dict[key] = {'filename': ele, 'director_name': elexx, 'doc_type': elexxx, 'count': count_}

In [ ]:
record_number_check_dict = record_number_check

In [ ]:
len(record_number_check_dict)

In [ ]:
def convert_dictionary_of_key_values_into_df(dict_):

    d = {'key': list(dict_.keys()),'values': list(dict_.values())}
    df_dict = pd.DataFrame(data = d)

    df_created = pd.concat([df_dict.drop(['values'], axis=1), df_dict['values'].apply(pd.Series)], axis=1)
    
    return df_created

In [ ]:
df_count_check_no_3Y = df_count_check[~(df_count_check['doc_type'] == '3Y')].reset_index(drop=True)

In [ ]:
df_count_check_no_3Y.sort_values(["count"], ascending=False).head()

In [ ]:
df_count_check_no_3Y[
    (df_count_check_no_3Y['count'] % 4 == 0)
    &
    (df_count_check_no_3Y['doc_type'] == '3Z')
].shape[0]

In [ ]:
df_count_check_no_3Y_greater_than = df_count_check_no_3Y[df_count_check_no_3Y['count'] > 5].reset_index(drop=True)
df_count_check_no_3Y_greater_than.shape[0]

df_count_check_no_3Y_greater_than = df_count_check_no_3Y_greater_than.sort_values(by=['count'], ascending = False)

df_count_check_no_3Y_greater_than['checked'] = int(0)
df_count_check_no_3Y_greater_than['result'] = ''
df_count_check_no_3Y_greater_than.head()

# loop to check number of records

In [ ]:
no_of_records = df_count_check_no_3Y_greater_than.shape[0]

index_list_grand = df_count_check_no_3Y_greater_than.index.tolist()

len_of_range_to_use = 42

for a in range(0, len_of_range_to_use):
    
    i = index_list_grand[a]
    
    if df_count_check_no_3Y_greater_than.loc[i, 'checked'] == 0:
        
        filename_ = df_count_check_no_3Y_greater_than.loc[i, 'filename']
        dir_name = df_count_check_no_3Y_greater_than.loc[i, 'director_name']
        doc_type = df_count_check_no_3Y_greater_than.loc[i, 'doc_type']
        count_ = df_count_check_no_3Y_greater_than.loc[i, 'count']
        
        # perform checks on indices of records
        
        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == dir_name)
            &
            (df_check_dates['document_type'] == doc_type)
        ]
        
        record_index_list = df_temp.index.tolist()
        
        record_index_list_sorted = sorted(record_index_list)
        
        index_len = len(record_index_list_sorted)
        
        start_int = record_index_list_sorted[0]
        end_int = record_index_list_sorted[index_len - 1]
        
        if start_int + index_len - 1 == end_int:
            
            length_test_result = 'TEST OK'
            
        else:
            
            length_test_result = '__ LOOKS WRONG __'
        
        openPDFFileFromFileName(filename_)
        
        result_string_first = '___________' + '\n' + str(a) + ' | ' + str(len_of_range_to_use)
        
        result_string_second = 'filename: ' + filename_ + '\n' + 'director: ' + dir_name + '\n' + 'doc_type: ' + doc_type + '\n' + 'count: ' + str(count_) + '\n' + 'record_index: ' + str(record_index_list)
        
        result_string_third = 'Length Test: ' + length_test_result
        
        result_string = result_string_first + '\n' + result_string_second + '\n' + result_string_third
        
        print(result_string)
        
        check_string = input('Ok?')
        
        if check_string == 'a':
            
            df_count_check_no_3Y_greater_than.at[i, 'result'] = 'OK'
            
        else:
            
            df_count_check_no_3Y_greater_than.at[i, 'result'] = 'ERROR'
            
        df_count_check_no_3Y_greater_than.at[i, 'checked'] = 1
        
        # SAVE DF IN LOOP

        filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))
        
        filename_stem = r'df_record_count_check_16JUL20'

        pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem + '.pickle'
        
        excel_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem + '.xlsx'

        saveFileAsPickle(pickle_filepath_and_filename, df_count_check_no_3Y_greater_than)
        
        saveAsExcelFile(excel_filepath_and_filename, df_count_check_no_3Y_greater_than)
        
print('Done!')
        
    

In [ ]:
df_count_check_no_3Y_greater_than.loc[212]

# save df_count_check

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_count_check_16JUL20_I.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_count_check)

# This is the one

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

doc_type_list = ['3X', '3Z']

record_number_check_string_test_dict = {}

for idx, ele in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    df_filtered_by_filename = df_check_dates[df_check_dates['filename'] == ele]
    
    dir_name_list = list(set(list(df_filtered_by_filename['director_name_round_four'])))
    
    for idxx, elexx in enumerate(dir_name_list):
        
        df_filtered_by_dir_name = df_filtered_by_filename[df_filtered_by_filename['director_name_round_four'] == elexx]
        
        for idxxx, elexxx in enumerate(doc_type_list):
            
            df_filtered_by_doc_type = df_filtered_by_dir_name[df_filtered_by_dir_name['document_type'] == elexxx]
            
            concat_strings_list = []
            
            if df_filtered_by_doc_type.shape[0] > 1: # only need to test where there are two or more rows (multiple of two)
            
                # perform index list check

                record_index_list = df_filtered_by_doc_type.index.tolist()

                record_index_list_sorted = sorted(record_index_list)

                index_len = len(record_index_list_sorted)
                
                for idxxxx, elexxxx in enumerate(record_index_list_sorted):
                    
                    string_1 = df_filtered_by_doc_type.loc[elexxxx, 'holder_name']
                    string_2 = df_filtered_by_doc_type.loc[elexxxx, 'interest_type']
                    string_3 = df_filtered_by_doc_type.loc[elexxxx, 'security_name_long']
                    string_4 = df_filtered_by_doc_type.loc[elexxxx, 'security_name_short']
                    string_5 = df_filtered_by_doc_type.loc[elexxxx, 'number']
                    string_6 = df_filtered_by_doc_type.loc[elexxxx, 'exercise_price']
                    string_7 = df_filtered_by_doc_type.loc[elexxxx, 'expiry_date']
                    
                    total_string = str(string_1) + str(string_2) + str(string_3) + str(string_4) + str(string_5) + str(string_6) + str(string_7)
                    
                    concat_strings_list.append(total_string)
                
                occurrences_list = collections.Counter(concat_strings_list)
                
                occurrences_list_unique_values = list(set(occurrences_list.values()))
                
#                 if ele == '5160892.pdf' and elexx == 'Andrew Tsang':
                    
#                     print(ele, elexx, elexxx, concat_strings_list, occurrences_list_unique_values)
                
                if len(occurrences_list_unique_values) == 1 and occurrences_list_unique_values[0] % 2 == 0:
                    
                    key = ele + r'_' + elexx.replace(' ', '_') + r'_' + elexxx

                    record_number_check_string_test_dict.setdefault(key, [])

                    record_number_check_string_test_dict[key] = {
                        'filename': ele,
                        'director_name': elexx,
                        'doc_type': elexxx,
                        'occurrences': occurrences_list_unique_values}

In [ ]:
len(record_number_check_string_test_dict)

In [ ]:
df_with_possible_duplication = convert_dictionary_of_key_values_into_df(record_number_check_string_test_dict)

df_with_possible_duplication['checked'] = 0
df_with_possible_duplication['result'] = ''
df_with_possible_duplication.head()

# loop to check for record count errors
1. need to use the Excel | pickle file 'df_with_possible_duplication_16JUL20_USE_THIS.xlsx' to ensure double counting | rework does not occur - ignore those records already checked

In [ ]:
# open existing processed df_with_possible_duplication_16JUL20_USE_THIS.xlsx file

file_name_ = r'C:\Users\Paul\backup_1Nov19\adjata\21_PICKLE_FILES\df_with_possible_duplication_16JUL20_USE_THIS.xlsx'

df_with_possible_duplication = openExcelFile(file_name_)

df_with_possible_duplication.drop('Unnamed: 0', axis = 1, inplace = True)
df_with_possible_duplication.reset_index(drop = True, inplace = True)

df_with_possible_duplication.head()

In [ ]:
# df_with_possible_duplication stats
df_with_possible_duplication.shape[0], df_with_possible_duplication['checked'].value_counts()

In [ ]:
no_of_records = df_with_possible_duplication.shape[0]

index_list_grand = df_with_possible_duplication.index.tolist()

len_of_range_to_use = 150

for a in range(0, len_of_range_to_use):
    
    i = index_list_grand[a]
    
    if df_with_possible_duplication.loc[i, 'checked'] == 0:
        
        filename_ = df_with_possible_duplication.loc[i, 'filename']
        dir_name = df_with_possible_duplication.loc[i, 'director_name']
        doc_type = df_with_possible_duplication.loc[i, 'doc_type']
        count_ = df_with_possible_duplication.loc[i, 'occurrences'][0]
        
        # perform checks on indices of records
        
        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == dir_name)
            &
            (df_check_dates['document_type'] == doc_type)
        ]
        
        record_index_list = df_temp.index.tolist()
        
        record_index_list_sorted = sorted(record_index_list)
        
        index_len = len(record_index_list_sorted)
        
        start_int = record_index_list_sorted[0]
        end_int = record_index_list_sorted[index_len - 1]
        
        if start_int + index_len - 1 == end_int:
            
            length_test_result = 'TEST OK'
            
        else:
            
            length_test_result = '__ LOOKS WRONG __'
        
        openPDFFileFromFileName(filename_)
        
        result_string_first = '___________' + '\n' + str(a) + ' | ' + str(len_of_range_to_use)
        
        result_string_second = 'filename: ' + filename_ + '\n' + 'director: ' + dir_name + '\n' + 'doc_type: ' + doc_type + '\n' + 'record_index: ' + str(len(record_index_list))
        
#         result_string_third = 'Length Test: ' + length_test_result

        result_string_third = 'OK: a, ERROR: e'
        
        result_string = result_string_first + '\n' + result_string_second + '\n' + result_string_third
        
        print(result_string)
        
        check_string = input('Ok?')
        
        if check_string.lower() == 'a':
            
            df_with_possible_duplication.at[i, 'result'] = 'OK'
            
        elif check_string.lower() == 'e':
            
            df_with_possible_duplication.at[i, 'result'] = 'ERROR'
            
        df_with_possible_duplication.at[i, 'checked'] = 1
        
        # SAVE DF IN LOOP

        filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))
        
        filename_stem = r'df_with_possible_duplication_20JUL20'

        pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem + '.pickle'
        
        excel_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem

        saveFileAsPickle(pickle_filepath_and_filename, df_with_possible_duplication)
        
        saveAsExcelFile(excel_filepath_and_filename, df_with_possible_duplication)
        
print('Done!')

# use duplication df to determine the rows to delete

In [ ]:
# open existing processed df_with_possible_duplication_16JUL20_USE_THIS.xlsx file

file_name_ = r'C:\Users\Paul\backup_1Nov19\adjata\21_PICKLE_FILES\df_with_possible_duplication_20JUL20.xlsx'

df_with_possible_duplication = openExcelFile(file_name_)

df_with_possible_duplication.drop('Column1', axis = 1, inplace = True)
df_with_possible_duplication.reset_index(drop = True, inplace = True)

df_with_possible_duplication.head()

In [ ]:
df_with_possible_duplication['action'] = ''
df_with_possible_duplication['action_checked'] = 0

In [ ]:
df_with_possible_duplication.head()

In [ ]:
records_to_correct_dict = {}

index_list = df_with_possible_duplication.index.tolist()

for idx, elex in enumerate(index_list):
    
    key = df_with_possible_duplication.loc[idx, 'key']
    filename_ = df_with_possible_duplication.loc[idx, 'filename']
    director_name = df_with_possible_duplication.loc[idx, 'director_name']
    doc_type = df_with_possible_duplication.loc[idx, 'doc_type']
    result_ = df_with_possible_duplication.loc[idx, 'result']
    
    if result_ == 'ERROR':
    
        records_to_correct_dict.setdefault(key, [])

        records_to_correct_dict[key] = {
            'filename': filename_,
            'director_name': director_name,
            'doc_type': doc_type,
            'result': result_
        }

In [ ]:
total_records_to_check = df_with_possible_duplication['result'].value_counts()['ERROR']
total_records_checked = df_with_possible_duplication['action_checked'].value_counts()[1]

In [ ]:
action_dict = {}

In [ ]:
# tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):

for k, v in tqdm_notebook(records_to_correct_dict.items(), total=len(records_to_correct_dict), desc='main_loop'):
    
    key_row_index = df_with_possible_duplication.index[df_with_possible_duplication['key'] == k].tolist()
    
    print(key_row_index)
    
    if df_with_possible_duplication.loc[key_row_index[0], 'action_checked'] == 0:
        
        total_records_to_check = df_with_possible_duplication['result'].value_counts()['ERROR']
        total_records_checked = df_with_possible_duplication['action_checked'].value_counts()[1]
    
        filename_ = v['filename']
        director_name = v['director_name']
        doc_type = v['doc_type']

        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == director_name)
            &
            (df_check_dates['document_type'] == doc_type)
        ]

        concat_strings_list = []

        # perform index list check

        record_index_list = df_temp.index.tolist()

        record_index_list_sorted = sorted(record_index_list)

        index_len = len(record_index_list_sorted)

        unique_strings_separate_index_dict = {}

        for idxxxx, elexxxx in enumerate(record_index_list_sorted):

            string_1 = df_temp.loc[elexxxx, 'holder_name']
            string_2 = df_temp.loc[elexxxx, 'interest_type']
            string_3 = df_temp.loc[elexxxx, 'security_name_long']
            string_4 = df_temp.loc[elexxxx, 'security_name_short']
            string_5 = df_temp.loc[elexxxx, 'number']
            string_6 = df_temp.loc[elexxxx, 'exercise_price']
            string_7 = df_temp.loc[elexxxx, 'expiry_date']

            total_string = str(string_1) + str(string_2) + str(string_3) + str(string_4) + str(string_5) + str(string_6) + str(string_7)

            key_ = elexxxx
            unique_strings_separate_index_dict.setdefault(key_, [])
            unique_strings_separate_index_dict[key_] = {'unique_strg': total_string}

            concat_strings_list.append(total_string)

        occurrences_list = collections.Counter(concat_strings_list)

        occurrences_list_unique_values = list(set(occurrences_list.values()))

        unique_strings_all_indices_dict = {}

        for kx in occurrences_list.keys():

            key_ = kx
            unique_strings_all_indices_dict.setdefault(key_, [])

            indices_to_match = []

            for kk, vv in unique_strings_separate_index_dict.items():

                if vv['unique_strg'] == kx:

                    indices_to_match.append(kk)

            unique_strings_all_indices_dict[key_] = {'indices': indices_to_match}

        # display the rows to be kept

        list_to_keep = []
        list_to_discard = []

        for kk, vv in unique_strings_all_indices_dict.items():

            temp_list = vv['indices']
            temp_list_sorted = sorted(temp_list)

            list_to_keep.append(temp_list_sorted[0])
            list_to_discard.append(temp_list_sorted[1:])

        string_to_display = ''
        string_header_to_display = ''

        for idd, eled in enumerate(list_to_keep):

            string_1 = df_check_dates.loc[eled, 'holder_name']
            string_2 = df_check_dates.loc[eled, 'interest_type']
            string_3 = df_check_dates.loc[eled, 'security_name_long']
            string_4 = df_check_dates.loc[eled, 'security_name_short']
            string_5 = df_check_dates.loc[eled, 'number']
            string_6 = df_check_dates.loc[eled, 'exercise_price']
            string_7 = df_check_dates.loc[eled, 'expiry_date']

            string_to_display = string_to_display + "\n" + r'_________________________'
            string_to_display = string_to_display + "\n" + 'holder_name: ' + str(string_1)
            string_to_display = string_to_display + "\n" + 'interest_type: ' + str(string_2)
            string_to_display = string_to_display + "\n" + 'security_name_long: ' + str(string_3)
            string_to_display = string_to_display + "\n" + 'security_name_short: ' + str(string_4)
            string_to_display = string_to_display + "\n" + 'number: ' + str(string_5)
            string_to_display = string_to_display + "\n" + 'exercise_price: ' + str(string_6)
            string_to_display = string_to_display + "\n" + 'expiry_date: ' + str(string_7)

        openPDFFileFromFileName(filename_)

        string_header_to_display = r'=========================' + "\n" + filename_ + ' ' + director_name + ' ' + doc_type + ' ' + str(total_records_checked + 1) + r' | ' + str(total_records_to_check)

        string_header_to_display = string_header_to_display + "\n" + string_to_display

        print(string_header_to_display)

        response_ = input('Do you want to go ahead?')

        DELETE_BOOL = False

        if response_ == 'ok':

            DELETE_BOOL = True

        elif response_ == 'no':

            DELETE_BOOL = False

        elif response_ == 'xx':

            DELETE_BOOL = False
            xxxx

        else:

            DELETE_BOOL = False
            xxxx

        # delete rows if DELETE_BOOL = True

        action_taken_dict = {True: 'ROWS DELETED', False: 'NO ROWS DELETED'}

    #     key_row_index = df_with_possible_duplication.index[df_with_possible_duplication['key'] == k].tolist()

        if DELETE_BOOL:

            for ii, ee in enumerate(list_to_discard):

                for iii, eee in enumerate(ee):

                    print('row dropped:', eee)
                    
                    if df_check_dates.loc[eee, 'director_name_round_four'] == director_name:

                        df_check_dates.drop([eee], inplace=True)
                        
                    else:
                        
                        print('____________ ERROR ____________')
                        XXXX

            df_with_possible_duplication.loc[key_row_index, 'action'] = action_taken_dict[DELETE_BOOL]
            df_with_possible_duplication.loc[key_row_index, 'action_checked'] = 1

        else:

            df_with_possible_duplication.loc[key_row_index, 'action'] = action_taken_dict[DELETE_BOOL]
            df_with_possible_duplication.loc[key_row_index, 'action_checked'] = 1

        # create dictionary of actions taken on rows

        keya = k

        action_dict.setdefault(keya, [])

        kept_dict = {}

        for idd, eled in enumerate(list_to_keep):

            key_ = eled
            kept_dict.setdefault(key_, [])

            kept_dict[key_] = {'index': key_, 'string_kept': unique_strings_separate_index_dict[key_], 'ACTION': action_taken_dict[DELETE_BOOL]}

        deleted_dict = {}

        for idd, eled in enumerate(list_to_discard):

            for iddd, eledd in enumerate(eled):

                key_ = eledd
                deleted_dict.setdefault(key_, [])

                deleted_dict[key_] = {'index': key_, 'string_deleted': unique_strings_separate_index_dict[key_], 'ACTION': action_taken_dict[DELETE_BOOL]}    

        action_dict[keya] = {'kept': kept_dict, 'deleted': deleted_dict}

        # save dictionary

        df_of_action_items = convert_dictionary_of_key_values_into_df(action_dict)

        # SAVE DF IN LOOP

        filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

        filename_stem = r'df_of_action_items_20JUL20'

        # save detailed description of action taken df

        pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem + '.pickle'

        excel_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem

        saveFileAsPickle(pickle_filepath_and_filename, df_of_action_items)

        saveAsExcelFile(excel_filepath_and_filename, df_of_action_items)   

        # save duplication file df

        filename_stem = r'df_with_possible_duplication_with_action_taken_20JUL20'

        pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem + '.pickle'

        excel_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + filename_stem

        saveFileAsPickle(pickle_filepath_and_filename, df_with_possible_duplication)

        saveAsExcelFile(excel_filepath_and_filename, df_with_possible_duplication)    

        print(filename_, director_name, doc_type, list_to_keep, list_to_discard)
    
print('All done!')

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_20JUL20_DEDUPLICATED.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

# ensure that when the interest_type (Direct | Indirect) is Direct, the holder_name is the same as director_name_round_four

In [ ]:
df_check_dates.head()

In [ ]:
filename_ = list(set(list(df_check_dates['filename'])))

doc_type_list = ['3X', '3Z']
interest_type_list = ['Direct', 'Indirect']

interest_type_check_dict = {}

for idx, elex in tqdm_notebook(enumerate(filenames_), total=len(filenames_), desc='main_loop'):
    
    df_filtered_by_filename = df_check_dates[
        (df_check_dates['filename'] == elex)
    ]
    
    dir_name_list = list(set(list(df_filtered_by_filename['director_name_round_four'])))
    
    for idxx, elexx in enumerate(dir_name_list):
        
        df_filtered_by_director = df_filtered_by_filename[
            (df_filtered_by_filename['director_name_round_four'] == elexx)
        ]
        
        for ii, ee in enumerate(doc_type_list):
        
            df_filtered_by_doc_type = df_filtered_by_director[
                (df_filtered_by_director['document_type'] == ee)
            ]
            
            if df_filtered_by_doc_type.shape[0] > 0:
                
                spec_index_list = df_filtered_by_doc_type.index.tolist()
                
                for i in range(0, len(spec_index_list)):
                    
                    index_ = spec_index_list[i]
                    
                    try:
            
                        number_int = float(df_filtered_by_doc_type.loc[index_, 'number'])

    #                     print(number_int, type(number_int))

                        if if_nan_return_True_otherwise_return_False(number_int):

                            number_int = 0

    #                     print(number_int)

                        if number_int > 0:

                            interest_type_ = df_filtered_by_doc_type.loc[index_, 'interest_type']
                            holder_name_ = df_filtered_by_doc_type.loc[index_, 'holder_name']

#                             if interest_type_ == 'Direct' and (holder_name_ != elexx): # TEST 1
                            if interest_type_ == 'Indirect' and (holder_name_ == elexx): # TEST 2

                                key = elex + '_' + elexx.replace(' ', '_') + '_' + ee + '_' + str(index_)
                                interest_type_check_dict.setdefault(key, [])

                                interest_type_check_dict[key] = {
                                    'filename': elex,
                                    'dir_name': elexx,
                                    'doc_type': ee,
                                    'interest_type': interest_type_,
                                    'holder_name': holder_name_,
                                    'number': number_int
                                }

                                print(interest_type_check_dict[key])
                                
                    except:
                        
                        key = elex + '_' + elexx.replace(' ', '_') + '_' + ee
                        interest_type_check_dict.setdefault(key, [])

                        interest_type_check_dict[key] = {
                            'filename': elex,
                            'dir_name': elexx,
                            'doc_type': ee,
                            'interest_type': 'ERROR',
                            'holder_name': 'ERROR',
                            'number': number_int
                        }

                        print(interest_type_check_dict[key])
                        
                        
                    
        
        

In [ ]:
interest_type_check_ACTION_dict = {}

for k, v in interest_type_check_dict.items():
    
    filename_ = v['filename']
    dir_name = v['dir_name']
    doc_type = v['doc_type']
    interest_type = v['interest_type']
    holder_name = v['holder_name']
    number_ = v['number']
    
    if dir_name.lower() == holder_name.lower() and interest_type == 'Indirect':
        
        action_ = 'CHANGE_TO_INDIRECT'
        
    else:
        
        action_ = ''
        
    key = k
    interest_type_check_ACTION_dict.setdefault(key, [])
    
    interest_type_check_ACTION_dict[key] = {
        'filename': filename_,
        'dir_name': dir_name,
        'doc_type': doc_type,
        'interest_type': interest_type,
        'holder_name': holder_name,
        'number': number_,
        'ACTION': action_
    }
    
    

In [ ]:
interest_type_check_ACTION_dict

In [ ]:
interest_type_check_MORE_ACTION_dict = {}

counter_ = 0

In [ ]:
for k, v in interest_type_check_ACTION_dict.items():
    
    if v['interest_type'] != 'ERROR' and v['ACTION'] != 'CHANGE_CASE':
        
        counter_ = counter_ + 1
        
        filename_ = v['filename']
        dir_name = v['dir_name']
        doc_type = v['doc_type']
        interest_type = v['interest_type']
        holder_name = v['holder_name']
        number_ = v['number']
        
        string_to_display = ''

        string_to_display = string_to_display + "\n" + r'_________________________'
        string_to_display = string_to_display + "\n" + 'filename: ' + str(filename_)
        string_to_display = string_to_display + "\n" + 'director_name: ' + str(dir_name)
        string_to_display = string_to_display + "\n" + 'document_type: ' + str(doc_type)
        string_to_display = string_to_display + "\n" + 'interest_type: ' + str(interest_type)
        string_to_display = string_to_display + "\n" + 'holder_name: ' + str(holder_name)
        string_to_display = string_to_display + "\n" + 'number: ' + str(number_)
            

        openPDFFileFromFileName(filename_)

        string_header_to_display = r'=========================' + ' ' + str(counter_) + ' | ' + str(len(interest_type_check_ACTION_dict))
        
        instructions_ = "cc: CHANGE_OVER_NAME | dd: CHANGE_TO_DIRECT | ii: CHANGE_TO_INDIRECT"

        string_header_to_display = string_header_to_display + "\n" + string_to_display + "\n" + instructions_

        print(string_header_to_display)

        response_ = input('What action?')
        
        indirect_holder_name = ''

        if response_ == 'cc':
            
            ACTION_ = 'CHANGE_OVER_NAME'
            
        elif response_ == 'dd':
            
            ACTION_ = 'CHANGE_TO_DIRECT'
            
        elif response_ == 'ii':
            
            ACTION_ = 'CHANGE_TO_INDIRECT'
            
            indirect_holder_name = input('What indirect holder name?')
            
        else:
            
            xxxxx
            
        key = k
        interest_type_check_MORE_ACTION_dict.setdefault(key, [])
        
        interest_type_check_MORE_ACTION_dict[key] = {
            'filename': filename_,
            'dir_name': dir_name,
            'doc_type': doc_type,
            'interest_type': interest_type,
            'holder_name': holder_name,
            'number': number_,
            'ACTION': ACTION_,
            'indirect holder_name': indirect_holder_name
        }
        
print('done!')
        
        
        
        
        

In [ ]:
interest_type_check_MORE_ACTION_dict['903276.pdf_Tonianne_Dwyer_3Z_45737']

In [ ]:
interest_type_check_MORE_ACTION_RESULT_dict = {}

In [ ]:
for k, v in interest_type_check_MORE_ACTION_dict.items():
    
    filename_ = v['filename']
    dir_name = v['dir_name']
    doc_type = v['doc_type']
    interest_type = v['interest_type']
    holder_name = v['holder_name']
    number_ = v['number']
    action_ = v['ACTION']
    indirect_holder_name = v['indirect holder_name']
    
    RESULT_BOOL = ''
    
    if action_ == 'CHANGE_OVER_NAME':
        
        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == dir_name)
            &
            (df_check_dates['document_type'] == doc_type)
            &
            (df_check_dates['interest_type'] == interest_type)
            &
            (df_check_dates['holder_name'] == holder_name)
            &
            (df_check_dates['number'] == number_)
        ]
        
#         print(df_temp.shape[0])
        
        index_list = df_temp.index.tolist()

        for idx, elex in enumerate(index_list):
            
            if df_check_dates.loc[elex, 'director_name_round_four'] == dir_name and df_check_dates.loc[elex, 'holder_name'] == holder_name:
                
                df_check_dates.at[elex, 'holder_name'] = dir_name
                
                string_to_display = 'holder_name: ' + df_check_dates.loc[elex, 'holder_name'] + "\n" + 'dir_name: ' + df_check_dates.loc[elex, 'director_name_round_four']
                
                print(string_to_display)
            
                RESULT_BOOL = 'CHANGE_MADE'
                
    elif action_ == 'CHANGE_TO_INDIRECT':
        
        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == dir_name)
            &
            (df_check_dates['document_type'] == doc_type)
            &
            (df_check_dates['interest_type'] == interest_type)
            &
            (df_check_dates['holder_name'] == holder_name)
            &
            (df_check_dates['number'] == number_)
        ]
        
        index_list = df_temp.index.tolist()

        for idx, elex in enumerate(index_list):
            
            if df_check_dates.loc[elex, 'director_name_round_four'] == dir_name and df_check_dates.loc[elex, 'holder_name'] == holder_name and df_check_dates.loc[elex, 'interest_type'] == interest_type:
                
                df_check_dates.at[elex, 'interest_type'] = 'Indirect'
                df_check_dates.at[elex, 'holder_name'] = indirect_holder_name
                
                string_to_display = 'holder_name: ' + df_check_dates.loc[elex, 'holder_name'] + "\n" + 'dir_name: ' + df_check_dates.loc[elex, 'director_name_round_four']
                
                print('Indirect', string_to_display, indirect_holder_name)
            
                RESULT_BOOL = 'CHANGE_MADE'
                
    elif action_ == 'CHANGE_TO_DIRECT':
        
        df_temp = df_check_dates[
            (df_check_dates['filename'] == filename_)
            &
            (df_check_dates['director_name_round_four'] == dir_name)
            &
            (df_check_dates['document_type'] == doc_type)
            &
            (df_check_dates['interest_type'] == interest_type)
            &
            (df_check_dates['holder_name'] == holder_name)
            &
            (df_check_dates['number'] == number_)
        ]
        
        index_list = df_temp.index.tolist()

        for idx, elex in enumerate(index_list):
            
            if df_check_dates.loc[elex, 'director_name_round_four'] == dir_name and df_check_dates.loc[elex, 'holder_name'] == holder_name and df_check_dates.loc[elex, 'interest_type'] == interest_type:
                
                df_check_dates.at[elex, 'interest_type'] = 'Direct'
                
                string_to_display = 'holder_name: ' + df_check_dates.loc[elex, 'holder_name'] + "\n" + 'dir_name: ' + df_check_dates.loc[elex, 'director_name_round_four']
                
                print(string_to_display)
            
                RESULT_BOOL = 'CHANGE_MADE'
        
            
    key = k
    interest_type_check_MORE_ACTION_RESULT_dict.setdefault(key, [])

    interest_type_check_MORE_ACTION_RESULT_dict[key] = {
        'filename': filename_,
        'dir_name': dir_name,
        'doc_type': doc_type,
        'interest_type': interest_type,
        'holder_name': holder_name,
        'number': number_,
        'ACTION': action_,
        'RESULT': RESULT_BOOL
    }
        
#         df_check_dates.at[elex, 'holder_name'] == dir_name

print('done!')

In [ ]:
# filename: 7355242.pdf
# director_name: Keith Raymond Smith
# document_type: 3X
# interest_type: Indirect
# holder_name: Keith Raymond Smith
# number: 80.0

filename_ = '7355242.pdf'
dir_name = 'Keith Raymond Smith'
doc_type = '3X'
# interest_type = 'Direct'
number_ = 80.0

df_temp = df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
    &
    (df_check_dates['document_type'] == doc_type)
#     &
#     (df_check_dates['interest_type'] == interest_type)
    &
    (df_check_dates['number'] == number_)
]

print(df_temp['holder_name'], df_temp['interest_type'])

In [ ]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_24JUL20_II.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

In [ ]:
for k, v in interest_type_check_MORE_ACTION_RESULT_dict.items():
    
    if v['ACTION'] == 'CHANGE_OVER_NAME' and v['RESULT'] == 'CHANGE_MADE':
        
        print(k, v)

# miscellaneous

In [ ]:
# 01881493.pdf Mark Hamish Lochtenberg 3Z

# filename_ = '656649.pdf'
dir_name = 'Simon David Yeo'
# dir_name = 'Russell Peter Kane'
# doc_type = '3Z'

df_test = df_check_dates[
#     (df_check_dates['filename'] == filename_)
#     &
    (df_check_dates['director_name_round_four'] == dir_name)
#     (df_check_dates['director_name_round_four'] == dir_name)
#     &
#     (df_check_dates['document_type'] == doc_type)
]

filename_list = list(set(list(df_test['filename'])))
dir_name_list = dir_name_old_list = list(set(list(df_test['director_name_round_four'])))
dir_name_old_list = list(set(list(df_test['director_name_round_four_old'])))
holder_name_list = list(set(list(df_test['holder_name'])))
index_list = df_test.index.tolist()
doc_type_list = list(set(list(df_test['document_type'])))

print(filename_list, dir_name_old_list, dir_name_list, holder_name_list, index_list, doc_type_list)

In [ ]:
for i in range(0, len(index_list)):
    
    index_ = index_list[i]
    
#     df_check_dates.at[index_, 'director_name_round_four'] = dir_name_old_list[0]

In [ ]:
simon_david_yeo_dict = {}

for idx, elex in enumerate(filename_list):
    
    df_test = df_check_dates[(df_check_dates['filename'] == elex)]
    
    doc_type_list = list(set(list(df_test['document_type'])))
    
    if '3X' in doc_type_list or '3Z' in doc_type_list:
    
        openPDFFileFromFileName(elex)

        wait_ = input('next')

In [ ]:
'656649.pdf': 'Simon David Yeo'
'6566502.pdf': 'Russell Peter Kane'

In [ ]:
openPDFFileFromFileName(filename_)

In [ ]:
# Notes from merging:

# -------------------------------------------
# 1. ('749904.pdf', '3X', 'Jason William Peterson')
# Appears in df_SINGLE_3X_.pickle but not in df_combined_adjusted_for_consolidated_ENTITY_NAMES_and_MANUAL_CHANGES_DIRECTOR_NAME_REVIEW_6JUL20.pickle
# This record must have manually been created by me in the Excel file when processing the review of 3X | 3Z files
# ACTION:
# Create Excel file out of source pickle file - add the records for ('749904.pdf', '3X', 'Jason William Peterson')
# and convert back to pickle file

# -------------------------------------------
# 2. ('838927.pdf', '3X', 'Allan Preston Beasley')
# Appears in df_SINGLE_3X_.pickle but not in df_combined_adjusted_for_consolidated_ENTITY_NAMES_and_MANUAL_CHANGES_DIRECTOR_NAME_REVIEW_6JUL20.pickle
# This record must have manually been created by me in the Excel file when processing the review of 3X | 3Z files
# ACTION:
# Create Excel file out of source pickle file - add the records for ('838927.pdf', '3X', 'Allan Preston Beasley')
# and convert back to pickle file

# check for null 'number' instances

In [ ]:
filename_list = list(set(list(df_check_dates['filename'])))

df_check_dates_filtered = df_check_dates[
    ~(df_check_dates['document_type'] == '3Y')
]

doc_type_list = ['3X', '3Z']

check_number_for_null_dict = {}

check_type = 'Indirect_check_for_null_number'

for idx, elex in tqdm_notebook(enumerate(filename_list), total=len(filename_list), desc='main_loop'):
    
    # filter by filename
    df_temp_I = df_check_dates_filtered[df_check_dates_filtered['filename'] == elex]
    
    director_name_list = list(set(list(df_temp_I['director_name_round_four'])))
    
    for idxx, elexx in enumerate(director_name_list):
        
        # filter by director name
        df_temp_II = df_temp_I[df_temp_I['director_name_round_four'] == elexx]
        
        for ii, ee in enumerate(doc_type_list):
            
            # filter by document type
            df_temp_III = df_temp_II[df_temp_II['document_type'] == ee]
        
            index_error_list = [] 
            
            index_list = df_temp_III.index.tolist()
            
            for aa, bb in enumerate(index_list):
            
                interest_type = df_check_dates.loc[bb, 'interest_type']
                holder_name = df_check_dates.loc[bb, 'holder_name']
                security_name_long = df_check_dates.loc[bb, 'security_name_long']
                security_name_short = df_check_dates.loc[bb, 'security_name_short']
                
                number_ = df_check_dates.loc[bb, 'number']
                exercise_price = df_check_dates.loc[bb, 'exercise_price']
                
                expiry_date = df_check_dates.loc[bb, 'expiry_date']
                
                if expiry_date == 0:
                    
                    expiry_date = 'NaN'
                
                try:
                    expiry_date = convert_df_cell_string_to_date_object(df_check_dates.loc[bb, 'expiry_date'])

#                     if interest_type == 'Indirect' and (len(str(holder_name)) > 3 and holder_name != elexx) and (number_ < 1 and ((exercise_price > 0 and len(str(expiry_date)) < 3) or (exercise_price == 0 and len(str(expiry_date)) > 3))):
                    if interest_type == 'Direct' and (len(str(holder_name)) > 3 and holder_name != elexx) and (number_ < 1):
#                     if interest_type == 'Direct' and (len(str(holder_name)) > 3 and holder_name != elexx) and (number_ < 1 and ((exercise_price > 0 and len(str(expiry_date)) < 3) or (exercise_price == 0 and len(str(expiry_date)) > 3))):

                        key = str(elex) + r'_' + str(elexx.replace(' ', '_')) + '_' + ee + '_' + str(bb)

                        check_number_for_null_dict.setdefault(key, [])

                        check_number_for_null_dict[key] = {
                            'check_type': check_type,
                            'filename': elex,
                            'director': elexx,
                            'doc_type': ee,
                            'index': bb,
                            'interest_type': interest_type,
                            'holder': holder_name,
                            'security_name_long': security_name_long,
                            'security_name_short': security_name_short,
                            'number': number_,
                            'exercise_price': exercise_price,
                            'expiry_date': expiry_date,
                            'TEST': 'OK'
                        }
                        
                except:
                    
                    key = str(elex) + r'_' + str(elexx.replace(' ', '_')) + '_' + ee + '_' + str(bb)
                    
                    check_number_for_null_dict.setdefault(key, [])

                    check_number_for_null_dict[key] = {
                        'check_type': check_type,
                        'filename': elex,
                        'director': elexx,
                        'doc_type': ee,
                        'index': bb,
                        'interest_type': interest_type,
                        'holder': holder_name,
                        'security_name_long': security_name_long,
                        'security_name_short': security_name_short,
                        'number': number_,
                        'exercise_price': exercise_price,
                        'expiry_date': 'ERROR',
                        'TEST': 'DATE_ERROR'
                    }
                
                # *******************************

                        
                    
                    

                        

In [ ]:
check_number_for_null_dict

In [ ]:
index_list_to_check = []

for k, v in check_number_for_null_dict.items():
    
    index_list_to_check.append(v['index'])

In [ ]:
index_ = index_list_to_check[2]   

filename_ = df_check_dates.loc[index_, 'filename']
dir_name = df_check_dates.loc[index_, 'director_name_round_four']
holder_ = df_check_dates.loc[index_, 'holder_name']
number_ = df_check_dates.loc[index_, 'number']

openPDFFileFromFileName(filename_)

print(index_, '|', filename_, '|', dir_name, '|', holder_, '|', number_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
]



In [ ]:
# df_check_dates.at[index_, 'number'] = 1898078
df_check_dates.at[index_, 'holder_name'] = 'Simon Lucien Tregoning'

In [141]:
# save df_check_dates.pickle file

filepath_filename_tuple = os.path.split(os.path.abspath(df_GRAND_SOURCE))

pickle_filepath_and_filename = filepath_filename_tuple[0] + u'\\' + 'df_GRAND_3X_3Y_3Z_file_MERGED_NAMES_24JUL20_VI.pickle'

saveFileAsPickle(pickle_filepath_and_filename, df_check_dates)

In [ ]:
for k, v in check_number_for_null_dict.items():
    
    if v['TEST'] == 'DATE_ERROR':
        
        print(k, v)

In [ ]:
# 432752.pdf_Christopher_Quirk_3Y_273833

filename_ = '432752.pdf'
dir_name = 'Christopher Quirk'
document_type = '3Y'

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
    &
    (df_check_dates['document_type'] == document_type)
]

# sandbox

In [ ]:
possible_errors_dictionary = r'C:\Users\Paul\backup_1Nov19\adjata\21_PICKLE_FILES\check_3X_3Z_dates_filename_and_coversheet_dict_13JUL20.pickle'

In [ ]:
possible_error_dict = open_pickle_file_as_df(possible_errors_dictionary)

In [ ]:
for k, v in possible_error_dict.items():
    
    if v['doc_type'] == '3X':
        print(k, v)

In [ ]:
filename_ = '607332.pdf'
dir_name = 'James Lewis Michael Malone'

filename_ = '01792355.pdf'
dir_name = 'Ashish Patel'

filename_ = '5160892.pdf'
dir_name = 'Andrew Tsang'

In [63]:
# 'filename': '883218.pdf',
#   'index': 525,
#   'holder_name': 'Peter Charles Constable & R J Constable ATF the Joint Account A/C'},

filename_ = '883218.pdf'
# dir_name = 'Vasile Timis'

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
#     &
#     (df_check_dates['director_name_round_four'] == dir_name)
]

df_check_dates.loc[525, 'holder_name']

2015 E:\announcements_COPIED\2015\10\06\883218.pdf


'Peter Charles Constable & R J Constable ATF the Joint Account A/C'

In [ ]:
df_check_dates.loc[4829, 'holder_name']

In [ ]:
df_check_dates.at[4829, 'holder_name'] = 'Langham Investments Pty Ltd ATF the Langham Family Trust'

In [ ]:
index_list = df_check_dates.index.tolist()

check_for_unwanted_int_dict = {}

column_to_test = 'director_name_round_four'

for idx, elex in tqdm_notebook(enumerate(index_list), total=len(index_list), desc='main_loop'):
    
    test_element = df_check_dates.loc[elex, column_to_test]
    
    try:
        
        test_this = int(test_element)
        
        key = df_check_dates.loc[elex, 'filename'] + '_' + str(elex)
        check_for_unwanted_int_dict.setdefault(key, [])
        
        check_for_unwanted_int_dict[key] = {
            'column_to_test': column_to_test,
            'filename': df_check_dates.loc[elex, 'filename'],
            'index': elex,
            'holder_name': test_element
        }
    
    except:
        
        continue

In [139]:
index_list = df_check_dates.index.tolist()

check_for_unwanted_x_dict = {}

column_to_test = 'change_nature'

for idx, elex in tqdm_notebook(enumerate(index_list), total=len(index_list), desc='main_loop'):
    
    test_element = df_check_dates.loc[elex, column_to_test]
    dir_name = df_check_dates.loc[elex, 'director_name_round_four']
    doc_type = df_check_dates.loc[elex, 'document_type']
    
    if isinstance(test_element, str) and 'xx' in test_element.lower() and test_element != dir_name and doc_type != '3Y':
        
        key = df_check_dates.loc[elex, 'filename'] + '_' + str(elex)
        check_for_unwanted_x_dict.setdefault(key, [])

        check_for_unwanted_x_dict[key] = {
            'column_to_test': column_to_test,
            'filename': df_check_dates.loc[elex, 'filename'],
            'index': elex,
            column_to_test: test_element
        }


In [140]:
len(check_for_unwanted_x_dict)

0

In [81]:
check_for_unwanted_x_dict

{'504325.pdf_9184': {'column_to_test': 'change_nature',
  'filename': '504325.pdf',
  'index': 9184,
  'change_nature': 'Held in Allco Finance Group lxx'},
 '502012.pdf_42863': {'column_to_test': 'change_nature',
  'filename': '502012.pdf',
  'index': 42863,
  'change_nature': 'Held in Allco Hybrid ixx Trust'},
 '301844.pdf_50204': {'column_to_test': 'change_nature',
  'filename': '301844.pdf',
  'index': 50204,
  'change_nature': 'held in singapore telecommunications lxx'},
 '301844.pdf_50205': {'column_to_test': 'change_nature',
  'filename': '301844.pdf',
  'index': 50205,
  'change_nature': 'held in singapore computer systems lxx'},
 '301844.pdf_50206': {'column_to_test': 'change_nature',
  'filename': '301844.pdf',
  'index': 50206,
  'change_nature': 'held in singapore technologies engineering lxx'},
 '301844.pdf_50207': {'column_to_test': 'change_nature',
  'filename': '301844.pdf',
  'index': 50207,
  'change_nature': 'held in singapore technologies engineering lxx'},
 '301844.

In [82]:
unwanted_x_key_list = []

for k, v in check_for_unwanted_x_dict.items():
    
    unwanted_x_key_list.append(k)

In [138]:
elex = unwanted_x_key_list[16]

filename_ = check_for_unwanted_x_dict[elex]['filename']
index_ = check_for_unwanted_x_dict[elex]['index']
change_nature = check_for_unwanted_x_dict[elex]['change_nature']
dir_name = df_check_dates.loc[index_, 'director_name_round_four']

print(filename_, '|', index_, '|', change_nature, '|', df_check_dates.loc[index_, 'change_nature'], '|', dir_name)

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
    &
    (df_check_dates['director_name_round_four'] == dir_name)
]
    
    

IndexError: list index out of range

In [136]:
df_check_dates.at[index_, 'change_nature'] = 'NaN'

In [124]:
df_check_dates[
    (df_check_dates['name_to_display'] == 'EBOS GROUP LIMITED')
#     &
#     (df_check_dates['director_name_round_four'] == dir_name)
    &
    (df_check_dates['document_type'] == '3Z')
    &
    (df_check_dates['event_date'].str.contains('2013'))
]

,filename,pdf_id,rec_id,file_date,document_type,rec_type0,rec_type,received_datetime,released_datetime,number_of_entities,exchange_id,ticker,entity_name,director_name,last_notice_date,cover_sheet_date,holder_name,interest_type,appointment_date,change_date,cessation_date,event_date,security_name_long,security_name_short,number,prior_number,acquired_number,disposed_number,after_number,change_nature,consideration,consideration_per_security,exercise_price,expiry_date,load_id,date_added,datetime,status,description,_merge_3X3Y3Z,_merge_load_log,unique_id,director_name_string,director_name_changed,director_name_changed_bool,ticker_reviewed_bool,entity_name_changed,entity_name_changed_bool,entity_name_reviewed_bool,director_name_final,director_name_final_bool,director_name_overall,director_name_overall_bool,alternative_name,director_name_round_two,director_name_round_two_bool,director_name_round_one,director_name_round_one_bool,entity_name_corrected,entity_name_final,entity_name_final_ticker_checked,entity_name_final_bool,orig_index_I,ASX_change_bool,ticker_changes,name_changes,last_ticker,last_name,ASX_change_dates,last_ASX_change_date,ticker_to_display,name_to_display,delisted_bool,delisted_date,orig_index_II,director_name_round_three,director_name_round_three_bool,director_name_round_four_old,director_name_round_four_bool,corrected_names_14Nov19_bool,director_name_round_four,director_name_corrected
38197,773334.pdf,329268,33632.0,2019-02-07,3Z,2010,NaN,2013-12-11 14:17:09,2013-12-11 14:17:09,1,0,EBO,EBOS GROUP LIMITED,S.C. Ottrey,2013-11-07,NaT,Sarah Christine Ottrey & Peter Calder Groves,Indirect,NaN,NaN,2013-12-11,2013-12-11,Ordinary Shares,Shares,7580.0,0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 07:38:12,2019-07-05 07:17:29,success,NaN,both,both,773334.pdf32926833632.03Z,S.C Ottrey,S C Ottrey,1,1,EBOS GROUP LIMITED,0,0,S C Ottrey,0,Sarah Ottrey,10,NaN,Sarah Ottrey,0,Sarah Ottrey,10,EBOS GROUP LIMITED,EBOS GROUP LIMITED,0,0,19485,0,NaN,NaN,NaN,NaN,NaT,NaT,EBO,EBOS GROUP LIMITED,0,NaN,19485,Sarah Ottrey,0,Sarah Ottrey,0,0,Sarah Ottrey,0
50671,773330.pdf,448961,51303.0,2019-02-07,3Z,2010,NaN,2013-12-11 14:08:17,2013-12-11 14:08:17,1,0,EBO,EBOS GROUP LIMITED,E.M. Coutts,2013-11-07,NaT,Como Nominees Limited,Indirect,NaN,NaN,2013-11-07,2013-11-07,Ordinary Shares,Shares,26066.0,0,0,0.0,0.0,0,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:32:19,2019-07-05 07:21:16,success,NaN,both,both,773330.pdf44896151303.03Z,E.M Coutts,E M Coutts,1,1,EBOS GROUP LIMITED,0,0,E M Coutts,0,Elizabeth Coutts,10,NaN,Elizabeth Coutts,0,Elizabeth Coutts,10,EBOS GROUP LIMITED,EBOS GROUP LIMITED,0,0,235898,0,NaN,NaN,NaN,NaN,NaT,NaT,EBO,EBOS GROUP LIMITED,0,NaN,235884,Elizabeth Coutts,0,Elizabeth Coutts,0,0,Elizabeth Coutts,0
63213,773328.pdf,449512,NaN,2019-02-07,3Z,2010,NaN,2013-12-11 14:06:18,2013-12-11 14:06:18,1,0,EBO,EBOS GROUP LIMITED,B.J. Wallace,2013-11-07,NaT,Whyte Adder No 3 Limited,Indirect,NaN,NaN,2013-12-11,2013-12-11,Ordinary Shares,Shares,6879236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:33:04,2019-07-05 07:18:04,document-only,Exception in Processiong 3Z Appendix Part-2,left_only,both,773328.pdf449512nan3Z,B.J Wallace,B J Wallace,1,1,EBOS GROUP LIMITED,0,0,B J Wallace,0,Brett Wallace,10,NaN,Brent Wallace,0,Brent Wallace,10,EBOS GROUP LIMITED,EBOS GROUP LIMITED,0,0,236899,0,NaN,NaN,NaN,NaN,NaT,NaT,EBO,EBOS GROUP LIMITED,0,NaN,236885,Brent Wallace,0,Brent Wallace,0,0,Brent Wallace,0
63214,773328.pdf,449512,NaN,2019-02-07,3Z,2010,NaN,2013-12-11 14:06:18,2013-12-11 14:06:18,1,0,EBO,EBOS GROUP LIMITED,B.J. Wallace,2013-11-07,NaT,Herpa Properties Limited,Indirect,NaN,NaN,2013-12-11,2013-12-11,Ordinary Shares,Shares,1302960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:33:04,2019-07-05 07:18:04,document-only,Exception in Processiong 3Z Appendix Part-2,left_only,both,773328.pdf449512nan3Z,B.J Wallace,B J Wallace

In [53]:
regex_pattern_ampersand = re.compile(r"&", re.I)
regex_pattern_multi_spaces = re.compile(r"\s{2,}", re.I)

test_str = "Plan B Trustees Limited ATF B & E Super Fund"

corrected_string_I = regex_pattern_ampersand.sub(r' & ', test_str)
corrected_string_II = regex_pattern_multi_spaces.sub(r' ', corrected_string_I)

print(corrected_string_II)

Plan B Trustees Limited ATF B & E Super Fund


In [62]:
regex_pattern_super_fund = re.compile(r"super\s+fund", re.I)
regex_pattern_ampersand = re.compile(r"&", re.I)
regex_pattern_multi_spaces = re.compile(r"\s{2,}", re.I)

for idx, elex in enumerate(unwanted_x_key_list):

    filename_ = check_for_unwanted_x_dict[elex]['filename']
    index_ = check_for_unwanted_x_dict[elex]['index']
    holder_name_ = check_for_unwanted_x_dict[elex]['holder_name']
    dir_name = df_check_dates.loc[index_, 'director_name_round_four']
    
    corrected_holder_name = regex_pattern_super_fund.sub('Superannuation Fund', holder_name_)
    corrected_string_I = regex_pattern_ampersand.sub(r' & ', corrected_holder_name)
    corrected_string_II = regex_pattern_multi_spaces.sub(r' ', corrected_string_I)
    
    if df_check_dates.loc[index_, 'holder_name'] == holder_name_ and df_check_dates.loc[index_, 'director_name_round_four'] == dir_name and df_check_dates.loc[index_, 'filename'] == filename_:
        
        df_check_dates.at[index_, 'holder_name'] = corrected_string_II
        
    else:
        
        print('ERROR', k, v)
    
    

In [80]:
# key_ = unwanted_x_key_list[5]

# filename_ = check_for_unwanted_x_dict[key_]['filename']

filename_ = '01863734.pdf'
# index_ = check_for_unwanted_x_dict[key_]['index']
# holder_name_ = check_for_unwanted_x_dict[key_]['holder_name']
# dir_name = df_check_dates.loc[index_, 'director_name_round_four']
# dir_name = 'Scott Frew'
# holder_name = 'Frew Nominees Pty Ltd'

# print(filename_, '|', index_, '|', holder_name_, '|', df_check_dates.loc[index_, 'holder_name'], '|', dir_name)

openPDFFileFromFileName(filename_)

df_check_dates[
    (df_check_dates['filename'] == filename_)
#     &
#     (df_check_dates['director_name_round_four'] == dir_name)
]


2017 E:\announcements_COPIED\2017\06\07\01863734.pdf


,filename,pdf_id,rec_id,file_date,document_type,rec_type0,rec_type,received_datetime,released_datetime,number_of_entities,exchange_id,ticker,entity_name,director_name,last_notice_date,cover_sheet_date,holder_name,interest_type,appointment_date,change_date,cessation_date,event_date,security_name_long,security_name_short,number,prior_number,acquired_number,disposed_number,after_number,change_nature,consideration,consideration_per_security,exercise_price,expiry_date,load_id,date_added,datetime,status,description,_merge_3X3Y3Z,_merge_load_log,unique_id,director_name_string,director_name_changed,director_name_changed_bool,ticker_reviewed_bool,entity_name_changed,entity_name_changed_bool,entity_name_reviewed_bool,director_name_final,director_name_final_bool,director_name_overall,director_name_overall_bool,alternative_name,director_name_round_two,director_name_round_two_bool,director_name_round_one,director_name_round_one_bool,entity_name_corrected,entity_name_final,entity_name_final_ticker_checked,entity_name_final_bool,orig_index_I,ASX_change_bool,ticker_changes,name_changes,last_ticker,last_name,ASX_change_dates,last_ASX_change_date,ticker_to_display,name_to_display,delisted_bool,delisted_date,orig_index_II,director_name_round_three,director_name_round_three_bool,director_name_round_four_old,director_name_round_four_bool,corrected_names_14Nov19_bool,director_name_round_four,director_name_corrected
35968,01863734.pdf,427025,48071.0,2019-02-05,3Z,2010,NaN,2017-06-07 10:22:01,2017-06-07 10:22:01,1,3,MIH,MNC Media Investment Limited,Peck Joo Tan,2017-06-07,NaT,Peck Joo Tan,Direct,NaN,NaN,2017-06-07,2017-06-07,Ordinary Shares,Shares,500000.0,0,0,0.0,0.0,Date of last notice estimated as not clear fro...,0.0,0.0,NaN,NaN,fc315c06-e5ea-46d7-b2d1-43dfc6481789,2019-07-05 10:00:25,2019-07-05 07:18:36,success,NaN,both,both,01863734.pdf42702548071.03Z,Peck Joo Tan,Peck Joo Tan,0,1,MNC Media Investment Limited,0,0,Peck Joo Tan,0,Peck Joo Tan,0,NaN,Peck Joo Tan,0,Peck Joo Tan,0,MNC MEDIA INVESTMENT LIMITED,MNC MEDIA INVESTMENT LIMITED,0,0,196578,1,"LTL, MIH","LINKETONE LIMITED, MNC MEDIA INVESTMENT LIMITED",MIH,MNC MEDIA INVESTMENT LIMITED,2014-07-10,2014-07-10,MIH,MNC MEDIA INVESTMENT LIMITED,0,NaN,196568,Peck Joo Tan,0,Peck Joo Tan,0,0,Peck Joo Tan,0


In [77]:
df_check_dates.at[35968, 'change_nature'] = 'Date of last notice estimated as not clear from document'